In [2]:
import numpy as np
import pulp as pl
from numpy.typing import NDArray
from typing import Literal
from scipy.linalg import lu

In [3]:
def to_standard_equality_form(
    type: Literal["min", "max"], 
    A: NDArray[np.float64], 
    b: NDArray[np.float64], 
    c: NDArray[np.float64], 
    restriction_types: list[str], 
    non_negative_variables: list[bool] = []
) -> tuple[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]]:
    """
    Converts a linear programming problem to an equivalent problem in standard equality form.
    Parameters:
    A : 2D array
        Coefficient matrix of the constraints.
    b : 1D array
        Right-hand side vector of the constraints.
    c : 1D array
        Coefficient vector of the objective function.
    restriction_types : list of str
        List indicating the type of each constraint ('<=', '>=', '=').
    non_negative_variables : list of bool, optional
        List indicating whether each variable is non-negative. If empty, all variables are assumed to be non-negative.
    """
    # Number of constraints and number of variables
    m, n = A.shape

    # Convert minimization to maximization
    if type == "min":
        c = -c
        
    # Impose non-negativity on variables
    count_free = 0
    for (i, non_negative) in enumerate(non_negative_variables):
        if not non_negative:
            curr_i = i + count_free
            # Adjusting the objective function
            c = np.insert(c, curr_i+1, -c[curr_i])
            
            # Adjusting the constraint matrix
            A = np.insert(A, curr_i+1, -A[:, curr_i], axis=1)
            n += 1
            count_free += 1
            
    # Convert inequalities to equalities
    for (i, restriction) in enumerate(restriction_types):
        if restriction == "<=":
            # Add slack variable
            slack = np.zeros((m, 1))
            slack[i, 0] = 1
            A = np.hstack((A, slack))
            c = np.hstack((c, [0]))
            n += 1
        elif restriction == ">=":
            # Add surplus variable
            surplus = np.zeros((m, 1))
            surplus[i, 0] = -1
            A = np.hstack((A, surplus))
            c = np.hstack((c, [0]))
            n += 1
        
    return A, b, c

In [4]:
def to_canonical_form(
    B: list[int],
    N: list[int],
    A: NDArray[np.float64], 
    b: NDArray[np.float64], 
    c: NDArray[np.float64], 
    z: np.float64
) -> tuple[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]]:
    """
    Converts a linear programming problem to an equivalent problem in canonical form.
    Parameters:
    B: list of int
        List of indices representing the basic variables.
    N: list of int
        List of indices representing the non-basic variables.
    A : 2D array
        Coefficient matrix of the constraints.
    b : 1D array
        Right-hand side vector of the constraints.
    c : 1D array
        Coefficient vector of the objective function.
    restriction_types : list of str
        List indicating the type of each constraint ('<=', '>=', '=').
    non_negative_variables : list of bool, optional
        List indicating whether each variable is non-negative. If empty, all variables are assumed to be non-negative.
    """
    m, n = A.shape

    # Extract the submatrix corresponding to the chosen columns
    Ab = A[:, B]
    
    Ab_inv = np.linalg.inv(Ab)
    A_canon = Ab_inv @ A
    b_canon = Ab_inv @ b # b_canon is the solution for the basic variables
    
    if np.all(b_canon >= 0):  # Check if the solution is feasible (non-negative)
        # Transform to canonical form
        y = np.linalg.solve(Ab.T, c[list(B)]) # y = (A_b ^T)^-1 * c_b
        z_canon = z + y @ b
        c_canon = c - y @ A
        x_canon = np.zeros(n)
        x_canon[list(B)] = b_canon
        return A_canon, x_canon, b_canon, c_canon, z_canon

    raise ValueError("No basic feasible solution found; the problem may be infeasible.")

In [5]:
def phase_two_simplex(
    type: Literal["min", "max"],
    B: list[int], 
    A: NDArray[np.float64], 
    b: NDArray[np.float64], 
    c: NDArray[np.float64],
    z: np.float64, 
    restriction_types: list[str], 
    non_negative_variables: list[bool] = []
):
    '''
    Solves a linear programming problem using the Simplex method.
    Parameters:
    type : str
        Type of the problem, either "min" for minimization or "max" for maximization.
    B: list of int, optional
        List of indices representing the basic variables.
    A : 2D array
        Coefficient matrix of the constraints.
    b : 1D array
        Right-hand side vector of the constraints.
    c : 1D array
        Coefficient vector of the objective function.
    z : 1D array
        Constant term in the objective function.
    restriction_types : list of str
        List indicating the type of each constraint ('<=', '>=', '=').
    non_negative_variables : list of bool, optional
        List indicating whether each variable is non-negative. If empty, all variables are assumed to be non-negative.
    Returns:
        A : 2D array
    '''
    
    m, n = A.shape
    N = [j for j in range(n) if j not in B]
    
    while True:        
        # Step 1: Convert to canonical form
        A, x, b, c, z = to_canonical_form(B, N, A, b, c, z)
        
        # Step 2: Reached optimal solution
        if c[N].max() <= 0:
            opt_value = z + c @ x
            return x, opt_value
        
        # Step 3: Select k such that c[k] > 0
        k = 0
        for i in N:
            if c[i] > 0:
                k = i
                break
        
        # Step 4: If A_k <= then stop (the problem is unbounded)
        if A[:, k].max() <= 0:
            raise ValueError("The problem is unbounded.")
        
        # Step 5: Determine the index r and the value t for the next iteration of the simplex method.
        ratios = []
        for i in range(len(b)):
            if A[i, k] > 0:
                ratios.append((b[i] / A[i, k], i))
        t, r = min(ratios, key=lambda x: x[0])
        
        # Step 6: Let l be the rth basis element
        l = B[r]
        
        # Step 7: Set B = B \ {l} ∪ {k} and N = N \ {k} ∪ {l}
        B[r] = k
        
        N.remove(k)
        N.append(l)

        # 2.6 Atualiza A, b, c, z (pivot)
        Ab = A[:, B]
        Ab_inv = np.linalg.inv(Ab)
        A = Ab_inv @ A
        b = Ab_inv @ b
        y = np.linalg.solve(Ab.T, c[B])
        z = z + y @ b
        c = c - y @ A


In [6]:
def phase_one_simplex(A: NDArray[np.float64], b: NDArray[np.float64]):
    """
    Simplex Phase I: finds an initial feasible basis for Ax = b, x >= 0.

    Returns:
        feasible_basis : list[int] or None if the LP is infeasible.
    """
    m, n = A.shape

    # Add artificial variables to each constraint
    I_art = np.eye(m)
    A1 = np.hstack((A, I_art))
    c1 = np.zeros(n + m)
    c1[n:] = 1.0  # Minimize the sum of artificial variables

    # Initial basis: artificial variables
    B = list(range(n, n + m))
    N = [j for j in range(n)]

    max_iter = 1000
    for _ in range(max_iter):
        # Compute inverse of the current basis matrix
        A_B = A1[:, B]
        try:
            A_B_inv = np.linalg.inv(A_B)
        except np.linalg.LinAlgError:
            return ValueError("Couldn't solve: Singular basis.")

        # Basic feasible solution
        x_B = A_B_inv @ b

        # Compute reduced costs
        c_B = c1[B]
        c_N = c1[N]
        y = c_B @ A_B_inv
        reduced_costs = c_N - y @ A1[:, N]

        # Check optimality (all reduced costs ≥ 0 → optimum)
        if np.all(reduced_costs >= -1e-10):
            # Verify if artificial variables remaining in basis are zero
            for i, j in enumerate(B):
                if j >= n and abs(x_B[i]) > 1e-10:
                    raise ValueError("No basic feasible solution found; the problem may be infeasible.")
            break

        # Choose entering variable (most negative reduced cost)
        enter_idx = np.argmin(reduced_costs)
        entering = N[enter_idx]

        # Compute direction vector
        direction = A_B_inv @ A1[:, entering]
        if np.all(direction <= 1e-10):
            ValueError("The problem is unbounded.") # (should not occur in phase one)

        # Minimum ratio test
        ratios = [(x_B[i] / direction[i], i) for i in range(m) if direction[i] > 1e-10]
        if not ratios:
            raise ValueError("No basic feasible solution found; the problem may be infeasible.")
        _, leave_pos = min(ratios, key=lambda x: (x[0], x[1]))
        leaving = B[leave_pos]

        # Update basis and non-basis sets
        B[leave_pos] = entering
        N[enter_idx] = leaving
        B.sort()
        N.sort()

    # Extract feasible basis for the original problem
    feasible_B = [j for j in B if j < n]

    # Complete the basis if necessary
    if len(feasible_B) < m:
        for j in range(n):
            if j not in feasible_B:
                trial_basis = feasible_B + [j]
                if np.linalg.matrix_rank(A[:, trial_basis]) == len(trial_basis):
                    feasible_B = trial_basis
                    if len(feasible_B) == m:
                        break

    if len(feasible_B) != m:
        raise ValueError("No basic feasible solution found; the problem may be infeasible.")
    
    return feasible_B


In [7]:
def solve_lp(
    type: Literal["min", "max"],
    A: NDArray[np.float64], 
    b: NDArray[np.float64], 
    c: NDArray[np.float64],
    z: np.float64, 
    restriction_types: list[str], 
    non_negative_variables: list[bool] = []
) -> tuple[NDArray[np.float64], float]:
    '''
    Solves a linear programming problem using the Simplex method.
    Parameters:
    type : str
        Type of the problem, either "min" for minimization or "max" for maximization.
    A : 2D array
        Coefficient matrix of the constraints.
    b : 1D array
        Right-hand side vector of the constraints.
    c : 1D array
        Coefficient vector of the objective function.
    z : 1D array
        Constant term in the objective function.
    restriction_types : list of str
        List indicating the type of each constraint ('<=', '>=', '=').
    non_negative_variables : list of bool, optional
        List indicating whether each variable is non-negative. If empty, all variables are assumed to be non-negative.
    Returns:
        A tuple containing:
        - x : 1D array
            Solution vector.
        - opt_value : float
            Optimal value of the objective function.
    '''
    try: 
        # Step 0: Convert to standard equality form first
        A, b, c = to_standard_equality_form(type, A, b, c, restriction_types, non_negative_variables)
        
        B = phase_one_simplex(A, b)
        
        if B is None:
            ValueError("The problem is infeasible.")
        
        # Step 1: Phase 2 - solve the original problem starting from the basic feasible solution found in Phase 1
        return phase_two_simplex(type, B, A, b, c, z, restriction_types, non_negative_variables)
    
    except ValueError as e:
        print(e)
        return None, None

# Soluções dos exercícios do livro A Gentle Introduction to Optimization

Modelagens feitas pelo colega Thiago Vinicius Alves de Oliveira


# **Seção 1.2**


## 1.2.1 - Nutrição


Neste problema desejamos saber qual combinação de alimentos disponíveis devemos comprar para cumprir cada um dos requisitos nutricionais diários mínimos enquanto buscamos minimizar o custo total (podendo comprar porções fracionárias).

Os alimentos que temos disponíveis são:

- Raw carrots
- Baked potatoes
- Wheat bread
- Cheddar cheese
- Peanut butter

Para cada alimento, temos as medidas de:

- Preço por porção
- Calorias por porção
- Gordura (g) por porção
- Proteína (g) por porção
- Carboidratos (g) por porção

Os requisitos nutricionais diários mínimos são:

- Calorias ≥ 2000
- Gordura ≥ 50 g
- Proteína ≥ 100 g
- Carboidratos ≥ 250 g

Na modelagem do problema, nossas variáveis serão:

$$x_1  = \text{Quantidade de Raw carrots}$$

$$x_2  = \text{Quantidade de Baked potatoes}$$

$$x_3  = \text{Quantidade de Wheat bread}$$

$$x_4 = \text{Quantidade de Cheddar cheese}$$

$$x_5 = \text{Quantidade de Peanut butter}$$

Para todas elas, temos a restrição $x_i \geq 0$.

As informações disponíveis sobre o preço, caloria, gordura, proteína e carboidrato de cada variável está disponível na tabela:

| Alimento       | Preço ($) | Calorias | Gordura (g) | Proteína (g) | Carboidratos (g) |
| -------------- | --------- | -------- | ----------- | ------------ | ---------------- |
| Raw carrots    | 0.14      | 23       | 0.1         | 0.6          | 6                |
| Baked potatoes | 0.12      | 171      | 0.2         | 3.7          | 30               |
| Wheat bread    | 0.20      | 65       | 0.0         | 2.2          | 13               |
| Cheddar cheese | 0.75      | 112      | 9.3         | 7.0          | 0                |
| Peanut butter  | 0.15      | 188      | 16.0        | 7.7          | 2                |

Assim, se desejamos minimizar o lucro, nossa função objetivo é:

$$\min Z = 0.14x_1 + 0.12x_2 + 0.20x_3 + 0.75x_4 + 0.15x_5$$

Sujeito às seguintes restrições

Calorias:

$$
23x_1 + 171x_2 + 65x_3 + 112x_4 + 188x_5 \geq 2000
$$

Gordura:

$$
0.1x_1 + 0.2x_2 + 0.0x_3 + 9.3x_4 + 16.0x_5 \geq 50
$$

Proteína:

$$
0.6x_1 + 3.7x_2 + 2.2x_3 + 7.0x_4 + 7.7x_5 \geq 100
$$

Carboidratos:

$$
6x_1 + 30x_2 + 13x_3 + 0x_4 + 2x_5 \geq 250
$$


In [20]:
import pulp as pl

def solve_nutrition_problem():
    # Criação do problema
    lp = pl.LpProblem("Nutrition_Optimization", pl.LpMinimize)

    # Variáveis de decisão (servings de cada alimento)
    x1 = pl.LpVariable("Raw_carrots", lowBound=0)
    x2 = pl.LpVariable("Baked_potatoes", lowBound=0)
    x3 = pl.LpVariable("Wheat_bread", lowBound=0)
    x4 = pl.LpVariable("Cheddar_cheese", lowBound=0)
    x5 = pl.LpVariable("Peanut_butter", lowBound=0)

    # Função objetivo (minimizar custo total)
    lp += 0.14*x1 + 0.12*x2 + 0.20*x3 + 0.75*x4 + 0.15*x5, "Total_Cost"

    # Restrições nutricionais
    lp += 23*x1 + 171*x2 + 65*x3 + 112*x4 + 188*x5 >= 2000, "Calories"
    lp += 0.1*x1 + 0.2*x2 + 0*x3 + 9.3*x4 + 16*x5 >= 50, "Fat"
    lp += 0.6*x1 + 3.7*x2 + 2.2*x3 + 7*x4 + 7.7*x5 >= 100, "Protein"
    lp += 6*x1 + 30*x2 + 13*x3 + 0*x4 + 2*x5 >= 250, "Carbohydrate"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    solution = {var.name: var.varValue for var in [x1, x2, x3, x4, x5]}
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_nutrition_problem()

({'Raw_carrots': 0.0,
  'Baked_potatoes': 7.7146691,
  'Wheat_bread': 0.0,
  'Cheddar_cheese': 0.0,
  'Peanut_butter': 9.2799642},
 2.3177549219999998)

In [9]:
c = np.array([0.14, 0.12, 0.20, 0.75, 0.15])

A = np.array([
    [23,   171,  65,  112,  188],   # Calories
    [0.1,  0.2,  0,   9.3,  16],    # Fat
    [0.6,  3.7,  2.2, 7,    7.7],   # Protein
    [6,    30,   13,  0,    2]      # Carbohydrate
])

b = np.array([2000, 50, 100, 250])

signs = np.array([">=", ">=", ">=", ">="])  # Todas são "≥"

# Vetor de booleanos para não-negatividade (todas as variáveis são ≥ 0)
nonneg = np.array([True, True, True, True, True])

z = 0

x, z = solve_lp("min", A, b, c, z, signs, nonneg)
x, z

(array([   0.        ,    7.71466905,    0.        ,    0.        ,
           9.27996422, 1063.84168157,  100.02236136,    0.        ,
           0.        ]),
 np.float64(-2.317754919499106))

## 1.2.2 - MUCOW


Neste problema, a empresa MUCOW possui dois rebanhos de vacas (Holstein e Jersey) que produzem leite com diferentes teores de gordura. O objetivo é determinar como usar o leite produzido por cada rebanho para fabricar diferentes produtos lácteos (leite desnatado, leite 2%, leite integral, creme de mesa e creme para chantilly) de modo a maximizar o lucro total, respeitando as disponibilidades de leite e as especificações de gordura de cada produto.

Dados dos rebanhos:

- Holstein: 500 litros/dia, 3.7% de gordura

- Jersey: 250 litros/dia, 4.9% de gordura

Cada produto tem um percentual específico de gordura que deve ser exatamente atendido e cada produto contribui com certo valor para o lucro.

| Produto        | Volume por embalagem (L) | % Gordura | Lucro por embalagem ($) |
| -------------- | ------------------------ | --------- | ----------------------- |
| Skimmed milk   | 2.0                      | 0%        | 0.10                    |
| 2% milk        | 2.0                      | 2%        | 0.15                    |
| Whole milk     | 2.0                      | 4%        | 0.20                    |
| Table cream    | 0.6                      | 15%       | 0.50                    |
| Whipping cream | 0.3                      | 45%       | 1.20                    |

Seja $x_{ij}$ = litros do leite do tipo i usado no produto j, onde:

$$\text{i = H (Holstein), J (Jersey)}$$

$$\text{j = 1 (Skimmed), 2 (2\% milk), 3 (Whole), 4 (Table cream), 5 (Whipping cream)}$$

Assim temos 10 variáveis:
$$x_{H1}, x_{H2}, x_{H3}, x_{H4}, x_{H5}, x_{J1}, x_{J2}, x_{J3}, x_{J4}, x_{J5}$$

E a nossa função objetivo será maximizar:

$$\max Z = 0.1 \cdot \frac{x_{H1} + x_{J1}}{2} + 0.15 \cdot \frac{x_{H2} + x_{J2}}{2} + 0.2 \cdot \frac{x_{H3} + x_{J3}}{2} + 0.5 \cdot \frac{x_{H4} + x_{J4}}{0.6} + 1.2 \cdot \frac{x_{H5} + x_{J5}}{0.3}$$

Sujeito às seguintes restrições:

**1. Disponibilidade de leite:**

Holstein: $x_{H1} + x_{H2} + x_{H3} + x_{H4} + x_{H5} \leq 500$

Jersey: $x_{J1} + x_{J2} + x_{J3} + x_{J4} + x_{J5} \leq 250$

**2. Balanceamento de gordura por produto:**

- Skimmed milk (0% gordura):
  $$0.037x_{H1} + 0.049x_{J1} = 0$$

- 2% milk:
  $$0.037x_{H2} + 0.049x_{J2} = 0.02(x_{H2} + x_{J2})$$

- Whole milk (4% gordura):
  $$0.037x_{H3} + 0.049x_{J3} = 0.04(x_{H3} + x_{J3})$$

- Table cream (15% gordura):
  $$0.037x_{H4} + 0.049x_{J4} = 0.15(x_{H4} + x_{J4})$$

- Whipping cream (45% gordura):
  $$0.037x_{H5} + 0.049x_{J5} = 0.45(x_{H5} + x_{J5})$$

**3. Por fim, a restrição de não negatividade**
$$x_{ij} \geq 0 \quad \forall i,j$$


In [10]:
def solve_mucow_problem():
    # Criação do problema (MAXIMIZAR lucro)
    lp = pl.LpProblem("MUCOW_Production", pl.LpMaximize)

    # Variáveis de decisão (quantidade de cada produto)
    x1 = pl.LpVariable("Skimmed_milk", lowBound=0)
    x2 = pl.LpVariable("Two_percent_milk", lowBound=0)
    x3 = pl.LpVariable("Whole_milk", lowBound=0)
    x4 = pl.LpVariable("Table_cream", lowBound=0)
    x5 = pl.LpVariable("Whipping_cream", lowBound=0)

    # Função objetivo (maximizar lucro total)
    lp += 0.1*x1 + 0.15*x2 + 0.2*x3 + 0.5*x4 + 1.2*x5, "Total_Profit"

    # Restrições
    # 1. Volume total disponível
    lp += 2*x1 + 2*x2 + 2*x3 + 0.6*x4 + 0.3*x5 <= 750, "Total_Volume"
    
    # 2. Balanço de gordura total
    lp += 0*x1 + 0.04*x2 + 0.08*x3 + 0.09*x4 + 0.135*x5 == 27.75, "Fat_Balance"
    
    # 3. Composição exata de gordura para cada produto
    lp += 0*x1 == 0, "Skimmed_milk_fat"           # 0% gordura
    lp += 0.04*x2 - 0.02*(2*x2) == 0, "Two_percent_fat"    # 2% gordura
    lp += 0.08*x3 - 0.04*(2*x3) == 0, "Whole_milk_fat"     # 4% gordura
    lp += 0.09*x4 - 0.15*(0.6*x4) == 0, "Table_cream_fat"  # 15% gordura
    lp += 0.135*x5 - 0.45*(0.3*x5) == 0, "Whipping_cream_fat" # 45% gordura

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    solution = {var.name: var.varValue for var in [x1, x2, x3, x4, x5]}
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_mucow_problem()

Status: Optimal


({'Skimmed_milk': 344.16667,
  'Two_percent_milk': 0.0,
  'Whole_milk': 0.0,
  'Table_cream': 0.0,
  'Whipping_cream': 205.55556},
 281.083339)

In [11]:
A2 = np.array([
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
    [0.037, 0, 0, 0, 0, 0.049, 0, 0, 0, 0],
    [0, 0.017, 0, 0, 0, 0, 0.029, 0, 0, 0],
    [0, 0, -0.003, 0, 0, 0, 0, 0.009, 0, 0],
    [0, 0, 0, -0.113, 0, 0, 0, 0, -0.101, 0],
    [0, 0, 0, 0, -0.413, 0, 0, 0, 0, -0.401]
])

b2 = np.array([500, 250, 0, 0, 0, 0, 0])
c2 = np.array([
    0.1/2,    # x_HS
    0.15/2,   # x_HT
    0.2/2,    # x_HW
    0.5/0.6,  # x_HC
    1.2/0.3,  # x_HWhip
    0.1/2,    # x_JS
    0.15/2,   # x_JT
    0.2/2,    # x_JW
    0.5/0.6,  # x_JC
    1.2/0.3   # x_JWhip
])
sinais2 = ['<=', '<=', '=', '=', '=', '=', '=']

x, z = solve_lp("max", A2, b2, c2, 0, sinais2)
x, z

No basic feasible solution found; the problem may be infeasible.


(None, None)

## 1.2.3 - Salários


Neste problema, a diretora da startup CO-Tech precisa determinar os salários de seus sete funcionários (Tom, Peter, Nina, Samir, Gary, Linda e Bob) para o próximo ano, satisfazendo um conjunto de restrições de preferências e relações salariais entre eles.

No **item (a)** o objetivo é encontrar uma alocação de salários que atenda a todas as exigências, minimizando primeiro o custo total da folha salarial

E depois, no **item (b)**, queremos minimizar o salário do funcionário mais bem pago.

Na modelagem do problema, nossas variáveis serão:

$$x_1 = \text{Salário de Tom}$$

$$x_2 = \text{Salário de Peter}$$

$$x_3 = \text{Salário de Nina}$$

$$x_4 = \text{Salário de Samir}$$

$$x_5 = \text{Salário de Gary}$$

$$x_6 = \text{Salário de Linda}$$

$$x_7 = \text{Salário de Bob}$$

Para todas elas, temos a restrição $x_i \geq 0$.

As condições impostas pelos funcionários são modeladas como:

1. Tom quer pelo menos \$20.000 ou ele vai sair
   $$x_1 \geq 20000$$

2. Peter quer ser pago pelo menos \$5.000 a mais que Tom
   $$x_2 \geq x_1 + 5000$$

3. Nina quer ser paga pelo menos \$5.000 a mais que Tom
   $$x_3 \geq x_1 + 5000$$

4. Samir quer ser pago pelo menos \$5.000 a mais que Tom
   $$x_4 \geq x_1 + 5000$$

5. Gary quer que seu salário seja pelo menos igual ao salário combinado de Tom e Peter
   $$x_5 \geq x_1 + x_2$$

6. Linda quer que seu salário seja \$200 a mais que Gary
   $$x_6 = x_5 + 200$$

7. O salário combinado de Nina e Samir deve ser pelo menos o dobro do salário combinado de Tom e Peter
   $$x_3 + x_4 \geq 2(x_1 + x_2)$$

8. O salário de Bob é pelo menos tão alto quanto o de Peter
   $$x_7 \geq x_2$$

9. O salário de Bob é pelo menos tão alto quanto o de Samir
   $$x_7 \geq x_4$$

10. O salário combinado de Bob e Peter deve ser pelo menos \$60.000
    $$x_7 + x_2 \geq 60000$$

11. Linda não deve ganhar mais dinheiro que o salário combinado de Bob e Tom
    $$x_6 \leq x_7 + x_1$$

No **item (a)** desejamos minimizar o custo total da folha de pagamento mensal

$$ \min Z = x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7$$


In [12]:
def solve_co_tech_salaries_min_total():
    # Criação do problema
    lp = pl.LpProblem("CO_Tech_Salaries_Min_Total", pl.LpMinimize)

    # Variáveis de decisão (salários de cada funcionário)
    Tom = pl.LpVariable("Tom", lowBound=0)
    Peter = pl.LpVariable("Peter", lowBound=0)
    Nina = pl.LpVariable("Nina", lowBound=0)
    Samir = pl.LpVariable("Samir", lowBound=0)
    Gary = pl.LpVariable("Gary", lowBound=0)
    Linda = pl.LpVariable("Linda", lowBound=0)
    Bob = pl.LpVariable("Bob", lowBound=0)

    # Função objetivo (minimizar custo total de salários)
    lp += Tom + Peter + Nina + Samir + Gary + Linda + Bob, "Total_Salary_Cost"

    # Restrições
    # Tom wants at least $20,000
    lp += Tom >= 20000, "Tom_min_salary"
    
    # Peter, Nina, Samir each want at least $5000 more than Tom
    lp += Peter - Tom >= 5000, "Peter_vs_Tom"
    lp += Nina - Tom >= 5000, "Nina_vs_Tom"
    lp += Samir - Tom >= 5000, "Samir_vs_Tom"
    
    # Gary wants at least as high as Tom + Peter
    lp += Gary >= Tom + Peter, "Gary_vs_Tom_Peter"
    
    # Linda wants her salary to be $200 more than Gary
    lp += Linda == Gary + 200, "Linda_vs_Gary"
    
    # Combined salary of Nina and Samir ≥ 2*(Tom + Peter)
    lp += Nina + Samir >= 2 * (Tom + Peter), "Nina_Samir_vs_Tom_Peter"
    
    # Bob's salary ≥ Peter and ≥ Samir
    lp += Bob >= Peter, "Bob_vs_Peter"
    lp += Bob >= Samir, "Bob_vs_Samir"
    
    # Combined salary of Bob and Peter ≥ $60,000
    lp += Bob + Peter >= 60000, "Bob_Peter_min_combined"
    
    # Linda ≤ combined salary of Bob and Tom
    lp += Linda <= Bob + Tom, "Linda_vs_Bob_Tom"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    employees = ["Tom", "Peter", "Nina", "Samir", "Gary", "Linda", "Bob"]
    variables = [Tom, Peter, Nina, Samir, Gary, Linda, Bob]
    
    solution = {}
    total_salary = 0
    print("\nOptimal Salaries:")
    for emp, var in zip(employees, variables):
        salary = var.varValue
        solution[emp] = salary
        total_salary += salary
        print(f"  {emp}: ${salary:,.2f}")
    
    print(f"\nTotal Salary Cost: ${total_salary:,.2f}")
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_co_tech_salaries_min_total()

Status: Optimal

Optimal Salaries:
  Tom: $20,000.00
  Peter: $25,000.00
  Nina: $65,000.00
  Samir: $25,000.00
  Gary: $45,000.00
  Linda: $45,200.00
  Bob: $35,000.00

Total Salary Cost: $260,200.00


({'Tom': 20000.0,
  'Peter': 25000.0,
  'Nina': 65000.0,
  'Samir': 25000.0,
  'Gary': 45000.0,
  'Linda': 45200.0,
  'Bob': 35000.0},
 260200.0)

In [13]:
# Variáveis: [Tom, Peter, Nina, Samir, Gary, Linda, Bob]
c = np.array([1, 1, 1, 1, 1, 1, 1])  # Minimizar soma total dos salários

# Matriz A de restrições
A = np.array([
    # Tom wants at least $20,000
    [1, 0, 0, 0, 0, 0, 0],
    
    # Peter, Nina, Samir each want at least $5000 more than Tom
    [-1, 1, 0, 0, 0, 0, 0],
    [-1, 0, 1, 0, 0, 0, 0],
    [-1, 0, 0, 1, 0, 0, 0],
    
    # Gary wants at least as high as Tom + Peter
    [-1, -1, 0, 0, 1, 0, 0],
    
    # Linda wants $200 more than Gary
    [0, 0, 0, 0, -1, 1, 0],
    
    # Nina + Samir ≥ 2*(Tom + Peter)
    [-2, -2, 1, 1, 0, 0, 0],
    
    # Bob ≥ Peter and Bob ≥ Samir
    [0, -1, 0, 0, 0, 0, 1],
    [0, 0, 0, -1, 0, 0, 1],
    
    # Bob + Peter ≥ $60,000
    [0, 1, 0, 0, 0, 0, 1],
    
    # Linda ≤ Bob + Tom
    [-1, 0, 0, 0, 0, 1, -1]
])

b = np.array([20000, 5000, 5000, 5000, 0, 200, 0, 0, 0, 60000, 0])

signs = np.array([">=", ">=", ">=", ">=", ">=", "=", ">=", ">=", ">=", ">=", "<="])

# Todas as variáveis são não-negativas (salários)
nonneg = np.array([True, True, True, True, True, True, True])

z = 0

x, z = solve_lp("min", A, b, c, z, signs, nonneg)
x, z

(array([20000., 25000., 55000., 35000., 45000., 45200., 35000.,     0.,
            0., 30000., 10000.,     0.,     0., 10000.,     0.,     0.,
         9800.]),
 np.float64(-260200.0))

No **item (b)** queremos minimizar o maior salário. Podemos representá-lo por meio de uma nova variável $x_8$, fazendo:

$$\min Z = x_8$$

Sujeito a

$$x_8 \geq x_i$$

para $i = 1, 2, \dots, 7$


In [21]:
prob_b = pl.LpProblem("Min_Max_Salary", pl.LpMinimize)

x1 = pl.LpVariable("x1", lowBound=0, cat='Continuous')
x2 = pl.LpVariable("x2", lowBound=0, cat='Continuous')
x3 = pl.LpVariable("x3", lowBound=0, cat='Continuous')
x4 = pl.LpVariable("x4", lowBound=0, cat='Continuous')
x5 = pl.LpVariable("x5", lowBound=0, cat='Continuous')
x6 = pl.LpVariable("x6", lowBound=0, cat='Continuous')
x7 = pl.LpVariable("x7", lowBound=0, cat='Continuous')
x8 = pl.LpVariable("x8", lowBound=0, cat='Continuous')

prob_b += x8, "Minimize_Max_Salary"

prob_b += x1 >= 20000, "Tom_min"
prob_b += x2 >= x1 + 5000, "Peter_min"
prob_b += x3 >= x1 + 5000, "Nina_min"
prob_b += x4 >= x1 + 5000, "Samir_min"
prob_b += x5 >= x1 + x2, "Gary_min"
prob_b += x6 == x5 + 200, "Linda_eq"
prob_b += x3 + x4 >= 2*(x1 + x2), "Nina_Samir_combined"
prob_b += x7 >= x2, "Bob_vs_Peter"
prob_b += x7 >= x4, "Bob_vs_Samir"
prob_b += x7 + x2 >= 60000, "Bob_Peter_combined"
prob_b += x6 <= x7 + x1, "Linda_vs_Bob_Tom"

for i, var in enumerate([x1, x2, x3, x4, x5, x6, x7], 1):
    prob_b += x8 >= var, f"Max_Constraint_{i}"

prob_b.solve()

print("Status:", pl.LpStatus[prob_b.status])
for v in prob_b.variables():
    print(v.name, "=", v.varValue)
print("Maior Salário =", pl.value(prob_b.objective))

Status: Optimal
x1 = 20000.0
x2 = 25000.0
x3 = 45200.0
x4 = 45200.0
x5 = 45000.0
x6 = 45200.0
x7 = 45200.0
x8 = 45200.0
Maior Salário = 45200.0


In [23]:
A4 = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0],
    [-1, 1, 0, 0, 0, 0, 0, 0],
    [-1, 0, 1, 0, 0, 0, 0, 0],
    [-1, 0, 0, 1, 0, 0, 0, 0],
    [-1, -1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, -1, 1, 0, 0],
    [-2, -2, 1, 1, 0, 0, 0, 0],
    [0, -1, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, -1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 1, 0],
    [-1, 0, 0, 0, 0, 1, -1, 0],
    [-1, 0, 0, 0, 0, 0, 0, 1],
    [0, -1, 0, 0, 0, 0, 0, 1],
    [0, 0, -1, 0, 0, 0, 0, 1],
    [0, 0, 0, -1, 0, 0, 0, 1],
    [0, 0, 0, 0, -1, 0, 0, 1],
    [0, 0, 0, 0, 0, -1, 0, 1],
    [0, 0, 0, 0, 0, 0, -1, 1]
])

b4 = np.array([20000, 5000, 5000, 5000, 0, 200, 0, 0, 0, 60000, 0, 0, 0, 0, 0, 0, 0, 0])
c4 = np.array([0, 0, 0, 0, 0, 0, 0, 1])
sinais4 = ['>=', '>=', '>=', '>=', '>=', '=', '>=', '>=', '>=', '>=', '<=', '>=', '>=', '>=', '>=', '>=', '>=', '>=']

x, z = solve_lp("min", A4, b4, c4, 0, sinais4)
x, z

(array([20000., 25000., 44800., 45200., 45000., 45200., 45200., 45200.,
            0.,     0., 19800., 20200.,     0.,     0., 20200.,     0.,
        10200., 20000., 25200., 20200.,   400.,     0.,   200.,     0.,
            0.]),
 np.float64(-45200.0))

## 1.2.5


Neste problema, a planta CRUD produz composto químico X em diferentes horários e precisa enviá-lo para a planta FRESHAIR para reciclagem, obedecendo a capacidade máxima de armazenamento de 1000 litros e sem deixar estoque até o fim do dia. O objetivo é determinar quanto enviar a cada hora para minimizar o custo total de reciclagem.

Na modelagem do problema, nossas variáveis serão:

$$x_1 = \text{Litros enviados às 10h}$$
$$x_2 = \text{Litros enviados às 11h}$$
$$x_3 = \text{Litros enviados às 12h}$$
$$x_4 = \text{Litros enviados às 13h}$$
$$x_5 = \text{Litros enviados às 14h}$$
$$x_6 = \text{Litros enviados às 15h}$$

Para todas elas, temos a restrição $x_i \geq 0$.

Também definimos variáveis de estoque:
$$s_1 = \text{Estoque às 10h (após produção 9-10h)}$$
$$s_2 = \text{Estoque às 11h (após produção 10-11h)}$$
$$s_3 = \text{Estoque às 12h (após produção 11-12h)}$$
$$s_4 = \text{Estoque às 13h (após produção 12-13h)}$$
$$s_5 = \text{Estoque às 14h (após produção 13-14h)}$$
$$s_6 = \text{Estoque às 15h (após produção 14-15h)}$$

Para todos os estoques, temos $0 \leq s_i \leq 1000$ e $s_6 = 0$ (sem estoque até o fim do dia).

As condições do problema são modeladas como:

1. Estoque inicial às 10h: produção das 9-10h
   $$s_1 = 300 - x_1$$

2. Estoque às 11h: estoque anterior + produção 10-11h - envio às 11h
   $$s_2 = s_1 + 240 - x_2$$

3. Estoque às 12h: estoque anterior + produção 11-12h - envio às 12h
   $$s_3 = s_2 + 600 - x_3$$

4. Estoque às 13h: estoque anterior + produção 12-13h - envio às 13h
   $$s_4 = s_3 + 200 - x_4$$

5. Estoque às 14h: estoque anterior + produção 13-14h - envio às 14h
   $$s_5 = s_4 + 300 - x_5$$

6. Estoque às 15h: estoque anterior + produção 14-15h - envio às 15h
   $$s_6 = s_5 + 900 - x_6$$

7. Sem estoque overnight
   $$s_6 = 0$$

8. Limite de capacidade de armazenamento
   $$0 \leq s_i \leq 1000 \quad \text{para } i = 1,2,3,4,5,6$$

A função objetivo é minimizar o custo total de reciclagem:

$$\min Z = 30x_1 + 40x_2 + 35x_3 + 45x_4 + 38x_5 + 50x_6$$


In [14]:
def solve_crud_chemical_problem():
    # Criação do problema
    lp = pl.LpProblem("CRUD_Chemical_Plant", pl.LpMinimize)

    # Variáveis de decisão (litros enviados a cada hora)
    x10 = pl.LpVariable("Send_10am", lowBound=0)
    x11 = pl.LpVariable("Send_11am", lowBound=0)
    x12 = pl.LpVariable("Send_12pm", lowBound=0)
    x13 = pl.LpVariable("Send_1pm", lowBound=0)
    x14 = pl.LpVariable("Send_2pm", lowBound=0)
    x15 = pl.LpVariable("Send_3pm", lowBound=0)

    # Função objetivo (minimizar custo total de reciclagem)
    lp += 30*x10 + 40*x11 + 35*x12 + 45*x13 + 38*x14 + 50*x15, "Total_Recycling_Cost"

    # Produção por hora
    production = [300, 240, 600, 200, 300, 900]  # 9-10, 10-11, 11-12, 12-1, 1-2, 2-3
    
    # Preços de reciclagem
    prices = [30, 40, 35, 45, 38, 50]  # 10am, 11am, 12pm, 1pm, 2pm, 3pm

    # Restrições de estoque máximo (não pode exceder 1000 litros a qualquer momento)
    # Estoque às 11am: (300 - x10) + 240 ≤ 1000
    lp += 300 - x10 + 240 <= 1000, "Stock_11am"
    
    # Estoque às 12pm: (300 - x10) + (240 - x11) + 600 ≤ 1000
    lp += 300 - x10 + 240 - x11 + 600 <= 1000, "Stock_12pm"
    
    # Estoque às 1pm: (300 - x10) + (240 - x11) + (600 - x12) + 200 ≤ 1000
    lp += 300 - x10 + 240 - x11 + 600 - x12 + 200 <= 1000, "Stock_1pm"
    
    # Estoque às 2pm: (300 - x10) + ... + (200 - x13) + 300 ≤ 1000
    lp += 300 - x10 + 240 - x11 + 600 - x12 + 200 - x13 + 300 <= 1000, "Stock_2pm"
    
    # Estoque às 3pm: (300 - x10) + ... + (300 - x14) + 900 ≤ 1000
    lp += 300 - x10 + 240 - x11 + 600 - x12 + 200 - x13 + 300 - x14 + 900 <= 1000, "Stock_3pm"

    # Restrição de não deixar estoque overnight (todo chemical X deve ser enviado)
    total_production = sum(production)
    lp += x10 + x11 + x12 + x13 + x14 + x15 == total_production, "No_Overnight_Stock"

    # Restrições de capacidade máxima de envio (não pode enviar mais do que o disponível)
    lp += x10 <= 300, "Max_Send_10am"
    lp += x10 + x11 <= 300 + 240, "Max_Send_11am"
    lp += x10 + x11 + x12 <= 300 + 240 + 600, "Max_Send_12pm"
    lp += x10 + x11 + x12 + x13 <= 300 + 240 + 600 + 200, "Max_Send_1pm"
    lp += x10 + x11 + x12 + x13 + x14 <= 300 + 240 + 600 + 200 + 300, "Max_Send_2pm"
    lp += x10 + x11 + x12 + x13 + x14 + x15 <= total_production, "Max_Send_3pm"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    
    times = ["10am", "11am", "12pm", "1pm", "2pm", "3pm"]
    variables = [x10, x11, x12, x13, x14, x15]
    
    solution = {}
    total_cost = 0
    print("\nOptimal Recycling Schedule:")
    for time, var, price in zip(times, variables, prices):
        liters = var.varValue
        cost = liters * price
        solution[time] = liters
        total_cost += cost
        print(f"  {time}: {liters:.1f} liters (${cost:,.2f})")
    
    print(f"\nTotal Recycling Cost: ${total_cost:,.2f}")
    
    # Verificar estoques
    print("\nInventory Levels:")
    inventory = 0
    for i in range(6):
        inventory += production[i]
        if i < 5:
            inventory -= variables[i].varValue
        stock_cost = f" (${inventory * prices[i]:.2f} if recycled now)" if i > 0 else ""
        print(f"  {9+i}-{10+i}am: {inventory:.1f} liters{stock_cost}")
    
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_crud_chemical_problem()

Status: Optimal

Optimal Recycling Schedule:
  10am: 300.0 liters ($9,000.00)
  11am: 0.0 liters ($0.00)
  12pm: 840.0 liters ($29,400.00)
  1pm: 0.0 liters ($0.00)
  2pm: 500.0 liters ($19,000.00)
  3pm: 900.0 liters ($45,000.00)

Total Recycling Cost: $102,400.00

Inventory Levels:
  9-10am: 0.0 liters
  10-11am: 240.0 liters ($9600.00 if recycled now)
  11-12am: 0.0 liters ($0.00 if recycled now)
  12-13am: 200.0 liters ($9000.00 if recycled now)
  13-14am: 0.0 liters ($0.00 if recycled now)
  14-15am: 900.0 liters ($45000.00 if recycled now)


({'10am': 300.0,
  '11am': 0.0,
  '12pm': 840.0,
  '1pm': 0.0,
  '2pm': 500.0,
  '3pm': 900.0},
 102400.0)

In [15]:
# Variáveis: [x1, x2, x3, x4, x5, x6] = litros enviados às 10am, 11am, 12pm, 1pm, 2pm, 3pm
c = np.array([30, 40, 35, 45, 38, 50])  # Custos de reciclagem por litro

# Matriz A de restrições (estoque máximo e não-negatividade implícita)
A = np.array([
    # Restrição de estoque máximo às 10am: 300 - x1 ≤ 1000 (sempre satisfeita)
    # Restrição de estoque máximo às 11am: (300 - x1) + 240 - x2 ≤ 1000
    [-1, -1, 0, 0, 0, 0],
    
    # Restrição de estoque máximo às 12pm: (300 - x1) + (240 - x2) + 600 - x3 ≤ 1000
    [-1, -1, -1, 0, 0, 0],
    
    # Restrição de estoque máximo às 1pm: (300 - x1) + ... + 200 - x4 ≤ 1000
    [-1, -1, -1, -1, 0, 0],
    
    # Restrição de estoque máximo às 2pm: (300 - x1) + ... + 300 - x5 ≤ 1000
    [-1, -1, -1, -1, -1, 0],
    
    # Restrição de estoque máximo às 3pm: (300 - x1) + ... + 900 - x6 ≤ 1000
    [-1, -1, -1, -1, -1, -1],
    
    # Restrição de não deixar estoque overnight: todo chemical X deve ser enviado
    [1, 1, 1, 1, 1, 1],
    
    # Restrições de capacidade máxima de envio (não pode enviar mais do que o estoque disponível)
    [1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 0],
    [1, 1, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1]
])

b = np.array([
    540,    # 1000 - (300 + 240) = 460 → -x1 - x2 ≤ 460 → -x1 - x2 ≤ 540 (corrigido)
    140,    # 1000 - (300 + 240 + 600) = -140 → -x1 - x2 - x3 ≤ -140
    -60,    # 1000 - (300 + 240 + 600 + 200) = -340 → -x1 - ... - x4 ≤ -340
    -340,   # 1000 - (300 + 240 + 600 + 200 + 300) = -640 → -x1 - ... - x5 ≤ -640
    -640,   # 1000 - (300 + 240 + 600 + 200 + 300 + 900) = -1540 → -x1 - ... - x6 ≤ -1540
    2540,   # Total produzido: 300+240+600+200+300+900 = 2540
    300,    # x1 ≤ 300
    540,    # x1 + x2 ≤ 540
    1140,   # x1 + x2 + x3 ≤ 1140
    1340,   # x1 + ... + x4 ≤ 1340
    1640,   # x1 + ... + x5 ≤ 1640
    2540    # x1 + ... + x6 ≤ 2540
])

signs = np.array(["<=", "<=", "<=", "<=", "<=", "=", "<=", "<=", "<=", "<=", "<=", "<="])

# Todas as variáveis são não-negativas
nonneg = np.array([True, True, True, True, True, True])

z = 0

x, z = solve_lp("min", A, b, c, z, signs, nonneg)
x, z

(array([ 300.,    0.,  840.,    0.,  500.,  900.,  840., 1280., 1080.,
        1300., 1900.,    0.,  240.,    0.,  200.,    0.,    0.]),
 np.float64(-102400.0))

## 1.2.7


Neste problema, temos fábricas que produzem uma commodity e lojas que demandam essa commodity, com dois hubs centrais (A e B) como intermediários no transporte. O objetivo é minimizar o custo total de transporte atendendo à produção das fábricas e à demanda das lojas.

Na modelagem do problema, nossas variáveis serão:

**Variáveis de fábricas para hubs:**
$$x_{iA} = \text{Unidades enviadas da fábrica } i \text{ para o hub A}$$
$$x_{iB} = \text{Unidades enviadas da fábrica } i \text{ para o hub B}$$

**Variáveis de hubs para lojas:**
$$y_{Aj} = \text{Unidades enviadas do hub A para a loja } j$$
$$y_{Bj} = \text{Unidades enviadas do hub B para a loja } j$$

Para todas elas, temos a restrição $x_{iA}, x_{iB}, y_{Aj}, y_{Bj} \geq 0$.

As condições do problema são modeladas como:

**1. Capacidade das fábricas:** Cada fábrica $i$ não pode enviar mais do que produz
$$x_{iA} + x_{iB} \leq u_i \quad \text{para } i = 1,\dots,m$$

**2. Demanda das lojas:** Cada loja $j$ deve receber exatamente sua demanda
$$y_{Aj} + y_{Bj} = \ell_j \quad \text{para } j = 1,\dots,n$$

**3. Conservação nos hubs:** O total que entra em cada hub deve igualar o total que sai
$$\sum_{i=1}^m x_{iA} = \sum_{j=1}^n y_{Aj} \quad \text{(hub A)}$$
$$\sum_{i=1}^m x_{iB} = \sum_{j=1}^n y_{Bj} \quad \text{(hub B)}$$

A função objetivo é minimizar o custo total de transporte:

$$\min Z = \sum_{i=1}^m (a_i x_{iA} + b_i x_{iB}) + \sum_{j=1}^n (a^{\prime}_j y_{Aj} + b^{\prime}_j y_{Bj})$$

onde:

- $a_i$: custo de transporte por unidade da fábrica $i$ para o hub A
- $b_i$: custo de transporte por unidade da fábrica $i$ para o hub B
- $a^{\prime}_j$: custo de transporte por unidade do hub A para a loja $j$
- $b^{\prime}_j$: custo de transporte por unidade do hub B para a loja $j$


In [16]:
def solve_transportation_hubs_problem(m=3, n=4):
    """
    Solve the transportation problem with hubs.
    
    Parameters:
    m: number of factories
    n: number of stores
    """
    # Criação do problema
    lp = pl.LpProblem("Transportation_With_Hubs", pl.LpMinimize)

    # Variáveis de decisão
    # Transporte das fábricas para os hubs
    factory_to_A = []
    factory_to_B = []
    for i in range(1, m+1):
        factory_to_A.append(pl.LpVariable(f"Factory{i}_to_A", lowBound=0))
        factory_to_B.append(pl.LpVariable(f"Factory{i}_to_B", lowBound=0))
    
    # Transporte dos hubs para as lojas
    A_to_store = []
    B_to_store = []
    for j in range(1, n+1):
        A_to_store.append(pl.LpVariable(f"A_to_Store{j}", lowBound=0))
        B_to_store.append(pl.LpVariable(f"B_to_Store{j}", lowBound=0))

    # Custos de transporte (valores exemplo)
    # Custos das fábricas para os hubs
    cost_to_A = [2, 4, 3]  # a1, a2, a3
    cost_to_B = [3, 1, 2]  # b1, b2, b3
    
    # Custos dos hubs para as lojas
    cost_A_to = [5, 2, 4, 3]  # a'1, a'2, a'3, a'4
    cost_B_to = [3, 6, 2, 1]  # b'1, b'2, b'3, b'4
    
    # Capacidades das fábricas (valores exemplo)
    factory_capacity = [100, 150, 200]  # u1, u2, u3
    
    # Demandas das lojas (valores exemplo)
    store_demand = [80, 120, 90, 110]  # ℓ1, ℓ2, ℓ3, ℓ4

    # Função objetivo (minimizar custo total de transporte)
    total_cost = 0
    for i in range(m):
        total_cost += cost_to_A[i] * factory_to_A[i] + cost_to_B[i] * factory_to_B[i]
    for j in range(n):
        total_cost += cost_A_to[j] * A_to_store[j] + cost_B_to[j] * B_to_store[j]
    
    lp += total_cost, "Total_Transportation_Cost"

    # Restrições de capacidade das fábricas
    for i in range(m):
        lp += factory_to_A[i] + factory_to_B[i] <= factory_capacity[i], f"Factory{i+1}_Capacity"

    # Restrições de demanda das lojas
    for j in range(n):
        lp += A_to_store[j] + B_to_store[j] >= store_demand[j], f"Store{j+1}_Demand"

    # Restrições de conservação nos hubs (entrada = saída)
    total_to_A = pl.lpSum(factory_to_A)
    total_from_A = pl.lpSum(A_to_store)
    lp += total_to_A == total_from_A, "Hub_A_Conservation"
    
    total_to_B = pl.lpSum(factory_to_B)
    total_from_B = pl.lpSum(B_to_store)
    lp += total_to_B == total_from_B, "Hub_B_Conservation"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    
    solution = {}
    print("\nOptimal Transportation Plan:")
    
    print("\nFrom Factories to Hubs:")
    total_cost_factories = 0
    for i in range(m):
        to_A = factory_to_A[i].varValue
        to_B = factory_to_B[i].varValue
        cost_A = to_A * cost_to_A[i]
        cost_B = to_B * cost_to_B[i]
        total_cost_factories += cost_A + cost_B
        solution[f"Factory{i+1}_to_A"] = to_A
        solution[f"Factory{i+1}_to_B"] = to_B
        print(f"  Factory {i+1}: {to_A:.1f} to A (${cost_A:.2f}), {to_B:.1f} to B (${cost_B:.2f})")
    
    print("\nFrom Hubs to Stores:")
    total_cost_stores = 0
    for j in range(n):
        from_A = A_to_store[j].varValue
        from_B = B_to_store[j].varValue
        cost_A = from_A * cost_A_to[j]
        cost_B = from_B * cost_B_to[j]
        total_cost_stores += cost_A + cost_B
        solution[f"A_to_Store{j+1}"] = from_A
        solution[f"B_to_Store{j+1}"] = from_B
        print(f"  Store {j+1}: {from_A:.1f} from A (${cost_A:.2f}), {from_B:.1f} from B (${cost_B:.2f})")
    
    print(f"\nTotal Transportation Cost: ${total_cost_factories + total_cost_stores:,.2f}")
    print(f"  - Factories to Hubs: ${total_cost_factories:,.2f}")
    print(f"  - Hubs to Stores: ${total_cost_stores:,.2f}")
    
    # Verificar conservação
    print(f"\nHub A Conservation: In = {sum(factory_to_A[i].varValue for i in range(m)):.1f}, "
          f"Out = {sum(A_to_store[j].varValue for j in range(n)):.1f}")
    print(f"Hub B Conservation: In = {sum(factory_to_B[i].varValue for i in range(m)):.1f}, "
          f"Out = {sum(B_to_store[j].varValue for j in range(n)):.1f}")
    
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_transportation_hubs_problem()

Status: Optimal

Optimal Transportation Plan:

From Factories to Hubs:
  Factory 1: 100.0 to A ($200.00), 0.0 to B ($0.00)
  Factory 2: 0.0 to A ($0.00), 150.0 to B ($150.00)
  Factory 3: 20.0 to A ($60.00), 130.0 to B ($260.00)

From Hubs to Stores:
  Store 1: 0.0 from A ($0.00), 80.0 from B ($240.00)
  Store 2: 120.0 from A ($240.00), 0.0 from B ($0.00)
  Store 3: 0.0 from A ($0.00), 90.0 from B ($180.00)
  Store 4: 0.0 from A ($0.00), 110.0 from B ($110.00)

Total Transportation Cost: $1,440.00
  - Factories to Hubs: $670.00
  - Hubs to Stores: $770.00

Hub A Conservation: In = 120.0, Out = 120.0
Hub B Conservation: In = 280.0, Out = 280.0


({'Factory1_to_A': 100.0,
  'Factory1_to_B': 0.0,
  'Factory2_to_A': 0.0,
  'Factory2_to_B': 150.0,
  'Factory3_to_A': 20.0,
  'Factory3_to_B': 130.0,
  'A_to_Store1': 0.0,
  'B_to_Store1': 80.0,
  'A_to_Store2': 120.0,
  'B_to_Store2': 0.0,
  'A_to_Store3': 0.0,
  'B_to_Store3': 90.0,
  'A_to_Store4': 0.0,
  'B_to_Store4': 110.0},
 1440.0)

In [17]:
import numpy as np

# Variáveis: [f1A, f1B, f2A, f2B, f3A, f3B, As1, As2, As3, As4, Bs1, Bs2, Bs3, Bs4]
# Onde: fiA = unidades da fábrica i para hub A, fiB = unidades da fábrica i para hub B
#       Asj = unidades do hub A para loja j, Bsj = unidades do hub B para loja j

m = 3  # número de fábricas
n = 4  # número de lojas

# Custos: [a1, b1, a2, b2, a3, b3, a'1, a'2, a'3, a'4, b'1, b'2, b'3, b'4]
c = np.array([2, 3, 4, 1, 3, 2, 5, 2, 4, 3, 3, 6, 2, 1])  # Custos exemplo

# Matriz A de restrições (9 restrições × 14 variáveis)
A = np.array([
    # Restrições de capacidade das fábricas (f1A + f1B ≤ u1, etc.) - 3 restrições
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    
    # Restrições de demanda das lojas (As1 + Bs1 ≥ ℓ1, etc.) - 4 restrições
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
    
    # Restrições de conservação nos hubs (entrada = saída) - 2 restrições
    [1, 0, 1, 0, 1, 0, -1, -1, -1, -1, 0, 0, 0, 0],  # Hub A
    [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, -1, -1, -1, -1]   # Hub B
])

# Vetor b (9 elementos - um para cada restrição)
b = np.array([100, 150, 200, 80, 120, 90, 110, 0, 0])  # Removido o elemento extra

signs = np.array(["<=", "<=", "<=", ">=", ">=", ">=", ">=", "=", "="])

# Todas as variáveis são não-negativas
nonneg = np.array([True] * (2*m + 2*n))

z = 0

x, z = solve_lp("min", A, b, c, z, signs, nonneg)
x, z

(array([100.,   0.,   0., 150.,  20., 130.,   0., 120.,   0.,   0.,  80.,
          0.,  90., 110.,   0.,   0.,  50.,   0.,   0.,   0.,   0.]),
 np.float64(-1440.0))

## 1.2.8 - Classificação (Healthy/Unhealthy)


Neste problema, temos observações de tecidos humanos saudáveis (matriz A) e não saudáveis (matriz B), e queremos encontrar um hiperplano que melhor separe essas duas classes maximizando a margem entre elas.

Na modelagem do problema, nossas variáveis serão:

$$a = \text{Vetor normal ao hiperplano (coeficientes)}$$
$$\alpha = \text{Limite para tecidos saudáveis}$$
$$\beta = \text{Limite para tecidos não saudáveis}$$

Para $a \in \mathbb{R}^n$, $\alpha \in \mathbb{R}$, $\beta \in \mathbb{R}$.

As **restrições** do problema são modeladas como:

1. **Tecidos saudáveis devem estar abaixo de $\alpha$:**

Note que cada linha $A_i$ representa uma observação saudável.

E queremos

$a^\top A_i \leq \alpha$ para todo $i = 1,\dots,m$.

Escrevendo na forma matricial temos:

$$A a \leq \alpha \mathbf{1}_m$$

Onde $\mathbf{1}_m$ é um vetor de uns de dimensão $m$

2. **Tecidos não saudáveis devem estar acima de $\beta$:**

O raciocínio é análogo ao que fizemos acima

$$B a \geq \beta \mathbf{1}_p$$

3. **Maximizar a margem entre os conjuntos:**

A distância entre os hiperplanos $a^\top x = \alpha$ e $a^\top x = \beta$ é proporcional a $(\beta - \alpha)/||a||$. Para evitar a não-linearidade introduzida pelo denominador $||a||$, podemos fixar $||a|| = 1$ ou usar uma normalização alternativa. Uma abordagem comum é maximizar simplesmente $\beta - \alpha$:

$$\max (\beta - \alpha)$$

A função objetivo é maximizar a margem:

$$\max Z = \beta - \alpha$$

Sujeito às restrições:
$$A a \leq \alpha \mathbf{1}_m$$
$$B a \geq \beta \mathbf{1}_p$$

Esta formulação pode permitir soluções ilimitadas se não for imposta alguma restrição adicional em $a$. Veja que se $(a, \alpha, \beta)$ for uma solução $k*(a, \alpha, \beta)$ também será para $k > 0$. Como mencionado anteriormente, podemos adicionar a restrição $|| a || = 1$, que não é linear.


In [18]:
def solve_max_margin_classification_fixed():
    """
    Solve the max margin classification problem with fixed, feasible data.
    """
    # Dados fixos e claramente separáveis
    # Healthy (A) - cluster em torno de (2,2)
    A = np.array([
        [1.8, 2.1],
        [2.2, 1.9],
        [1.9, 2.3],
        [2.1, 1.8],
        [2.0, 2.0]
    ])

    # Unhealthy (B) - cluster em torno de (-2,-2)  
    B = np.array([
        [-2.1, -1.9],
        [-1.8, -2.2],
        [-2.2, -1.8],
        [-1.9, -2.1],
        [-2.0, -2.0]
    ])
    
    m, n = A.shape
    p = B.shape[0]
    
    print("Healthy data (A):")
    print(A)
    print("\nUnhealthy data (B):")
    print(B)
    print()
    
    # Criação do problema (MAXIMIZAR margem)
    lp = pl.LpProblem("Max_Margin_Classification", pl.LpMaximize)

    # Variáveis de decisão
    a1 = pl.LpVariable("a1", cat='Continuous')
    a2 = pl.LpVariable("a2", cat='Continuous')
    alpha = pl.LpVariable("alpha", cat='Continuous')
    beta = pl.LpVariable("beta", cat='Continuous')
    gamma = pl.LpVariable("gamma", cat='Continuous')  # Margem = alpha - beta

    # Função objetivo (maximizar a margem)
    lp += gamma, "Maximize_Margin"

    # Restrições para observações healthy: a^T x ≤ α
    for i in range(m):
        constraint_expr = a1 * A[i, 0] + a2 * A[i, 1]
        lp += constraint_expr <= alpha, f"Healthy_Constraint_{i}"

    # Restrições para observações unhealthy: a^T x ≥ β  
    for i in range(p):
        constraint_expr = a1 * B[i, 0] + a2 * B[i, 1]
        lp += constraint_expr >= beta, f"Unhealthy_Constraint_{i}"

    # Restrição que define a margem: γ = α - β
    lp += gamma == alpha - beta, "Margin_Definition"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    
    # Extrair valores das variáveis
    a1_val = a1.varValue
    a2_val = a2.varValue
    alpha_val = alpha.varValue
    beta_val = beta.varValue
    gamma_val = gamma.varValue
    
    a_solution = np.array([a1_val, a2_val])
    
    solution = {
        'a': a_solution,
        'alpha': alpha_val,
        'beta': beta_val,
        'gamma': gamma_val
    }
    
    print(f"\n=== OPTIMAL SOLUTION ===")
    print(f"  a = [{a1_val:.4f}, {a2_val:.4f}]")
    print(f"  α = {alpha_val:.4f}")
    print(f"  β = {beta_val:.4f}")
    print(f"  γ = α - β = {gamma_val:.4f} (margin)")
    
    # Verificar restrições
    print(f"\n=== CONSTRAINT VERIFICATION ===")
    
    # Verificar observações healthy
    print("Healthy observations (should be ≤ α):")
    healthy_ok = True
    for i in range(m):
        value = np.dot(a_solution, A[i])
        status = "✓" if value <= alpha_val + 1e-6 else "✗"
        print(f"  {status} A[{i}]: a^T x = {value:.4f} ≤ α = {alpha_val:.4f}")
        if value > alpha_val + 1e-6:
            healthy_ok = False
    
    # Verificar observações unhealthy
    print("\nUnhealthy observations (should be ≥ β):")
    unhealthy_ok = True
    for i in range(p):
        value = np.dot(a_solution, B[i])
        status = "✓" if value >= beta_val - 1e-6 else "✗"
        print(f"  {status} B[{i}]: a^T x = {value:.4f} ≥ β = {beta_val:.4f}")
        if value < beta_val - 1e-6:
            unhealthy_ok = False
    
    if healthy_ok and unhealthy_ok:
        print(f"\n✓ All constraints satisfied!")
    else:
        print(f"\n✗ Some constraints violated!")
    
    # Mostrar hiperplanos
    print(f"\n=== DECISION BOUNDARIES ===")
    print(f"  Healthy region: a^T x ≤ {alpha_val:.4f}")
    print(f"  Unhealthy region: a^T x ≥ {beta_val:.4f}")
    print(f"  Margin width: {gamma_val:.4f}")
    
    # Classificar alguns pontos de teste
    print(f"\n=== TEST POINT CLASSIFICATION ===")
    test_points = [
        [3.0, 3.0],   # Deve ser healthy
        [-3.0, -3.0], # Deve ser unhealthy  
        [0.0, 0.0]    # Na região de margem
    ]
    
    for point in test_points:
        value = np.dot(a_solution, point)
        if value <= alpha_val:
            classification = "HEALTHY"
        elif value >= beta_val:
            classification = "UNHEALTHY"
        else:
            classification = "IN MARGIN"
        print(f"  Point {point}: a^T x = {value:.4f} → {classification}")
    
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_max_margin_classification_fixed()

Healthy data (A):
[[1.8 2.1]
 [2.2 1.9]
 [1.9 2.3]
 [2.1 1.8]
 [2.  2. ]]

Unhealthy data (B):
[[-2.1 -1.9]
 [-1.8 -2.2]
 [-2.2 -1.8]
 [-1.9 -2.1]
 [-2.  -2. ]]

Status: Unbounded

=== OPTIMAL SOLUTION ===
  a = [0.0000, 0.0000]
  α = 0.0000
  β = 0.0000
  γ = α - β = 0.0000 (margin)

=== CONSTRAINT VERIFICATION ===
Healthy observations (should be ≤ α):
  ✓ A[0]: a^T x = 0.0000 ≤ α = 0.0000
  ✓ A[1]: a^T x = 0.0000 ≤ α = 0.0000
  ✓ A[2]: a^T x = 0.0000 ≤ α = 0.0000
  ✓ A[3]: a^T x = 0.0000 ≤ α = 0.0000
  ✓ A[4]: a^T x = 0.0000 ≤ α = 0.0000

Unhealthy observations (should be ≥ β):
  ✓ B[0]: a^T x = 0.0000 ≥ β = 0.0000
  ✓ B[1]: a^T x = 0.0000 ≥ β = 0.0000
  ✓ B[2]: a^T x = 0.0000 ≥ β = 0.0000
  ✓ B[3]: a^T x = 0.0000 ≥ β = 0.0000
  ✓ B[4]: a^T x = 0.0000 ≥ β = 0.0000

✓ All constraints satisfied!

=== DECISION BOUNDARIES ===
  Healthy region: a^T x ≤ 0.0000
  Unhealthy region: a^T x ≥ 0.0000
  Margin width: 0.0000

=== TEST POINT CLASSIFICATION ===
  Point [3.0, 3.0]: a^T x = 0.0000 → H

({'a': array([0., 0.]), 'alpha': 0.0, 'beta': 0.0, 'gamma': 0.0}, 0.0)

In [19]:
import numpy as np

# Dados fixos e factíveis
# Healthy (A) - cluster em torno de (2,2)
A = np.array([
    [1.8, 2.1],
    [2.2, 1.9],
    [1.9, 2.3],
    [2.1, 1.8],
    [2.0, 2.0]
])

# Unhealthy (B) - cluster em torno de (-2,-2)  
B = np.array([
    [-2.1, -1.9],
    [-1.8, -2.2],
    [-2.2, -1.8],
    [-1.9, -2.1],
    [-2.0, -2.0]
])

m, n = A.shape
p, n2 = B.shape

# Variáveis: [a1, a2, α, β, γ] onde γ = α - β (margem a ser maximizada)
c = np.zeros(n + 3)
c[-1] = -1  # Maximizar γ = α - β (minimizar -γ)

# Matriz de restrições
total_constraints = m + p + 1  # m para A + p para B + 1 para γ = α - β
A_lp = np.zeros((total_constraints, n + 3))

# Restrições para A (healthy): a^T x_i ≤ α
for i in range(m):
    A_lp[i, :n] = A[i]      # Coeficientes de a
    A_lp[i, n] = -1         # Coeficiente de -α
    A_lp[i, n+1] = 0        # Coeficiente de β
    A_lp[i, n+2] = 0        # Coeficiente de γ

# Restrições para B (unhealthy): a^T x_j ≥ β → -a^T x_j + β ≤ 0
for j in range(p):
    A_lp[m + j, :n] = -B[j]  # Coeficientes de -a
    A_lp[m + j, n] = 0       # Coeficiente de α
    A_lp[m + j, n+1] = 1     # Coeficiente de β  
    A_lp[m + j, n+2] = 0     # Coeficiente de γ

# Restrição adicional: γ = α - β → α - β - γ = 0
A_lp[-1, n] = 1      # α
A_lp[-1, n+1] = -1   # -β
A_lp[-1, n+2] = -1   # -γ

b = np.zeros(total_constraints)  # Todas as desigualdades são ≤ 0, mais a igualdade

signs = np.array(["<="] * (m + p) + ["="])

# Variáveis livres (sem restrição de não-negatividade)
nonneg = np.array([False] * (n + 3))

z = 0

x, z = solve_lp("min", A_lp, b, c, z, signs, nonneg)
x, z

The problem is unbounded.


(None, None)

# **Seção 1.3**


## 1.3.1


Nesse problema, temos um veículo com $3.6 \text{m}^3$ de capacidade e 7 tipos de itens disponíveis, cada um com volume e valor diferentes. Podemos levar no máximo uma unidade de cada item. O objetivo é maximizar valor total sem exceder a capacidade do veículo.

**Função objetivo:**
Maximizar o valor total:
$$\max Z = \sum_{i=1}^7 n_i \cdot x_i$$

**Restrição de volume:**
$$\sum_{i=1}^7 v_i \cdot x_i \leq 3.6$$

**Restrição de integralidade:**
$$x_i \in \{0,1\} \quad \text{para } i = 1,2,\ldots,7$$

Seja $x_i$ uma variável binária que indica se levamos o item tipo i:

$$
x_i = \begin{cases}
1 & \text{se levamos o item tipo i} \\
0 & \text{caso contrário}
\end{cases} \quad \text{para } i = 1,2,\ldots,7
$$

Conhecendo os valores dos itens, queremos maximar a seguinte função objetivo:

$$\max Z = 250x_1 + 300x_2 + 500x_3 + 700x_4 + 750x_5 + 900x_6 + 950x_7$$

Sujeitos a restrição de volume:

$$0.55x_1 + 0.6x_2 + 0.7x_3 + 0.75x_4 + 0.85x_5 + 0.9x_6 + 0.95x_7 \leq 3.6$$


In [24]:
prob = pl.LpProblem("Problema_do_Veiculo", pl.LpMaximize)

x1 = pl.LpVariable("x1", cat='Binary')  # Item 1
x2 = pl.LpVariable("x2", cat='Binary')  # Item 2
x3 = pl.LpVariable("x3", cat='Binary')  # Item 3
x4 = pl.LpVariable("x4", cat='Binary')  # Item 4
x5 = pl.LpVariable("x5", cat='Binary')  # Item 5
x6 = pl.LpVariable("x6", cat='Binary')  # Item 6
x7 = pl.LpVariable("x7", cat='Binary')  # Item 7

prob += 250*x1 + 300*x2 + 500*x3 + 700*x4 + 750*x5 + 900*x6 + 950*x7

prob += 0.55*x1 + 0.6*x2 + 0.7*x3 + 0.75*x4 + 0.85*x5 + 0.9*x6 + 0.95*x7 <= 3.6

prob.solve()

print("Status:", pl.LpStatus[prob.status])
print("Valor Total =", pl.value(prob.objective))
print("Volume Utilizado =",
      0.55*x1.varValue + 0.6*x2.varValue + 0.7*x3.varValue +
      0.75*x4.varValue + 0.85*x5.varValue + 0.9*x6.varValue + 0.95*x7.varValue)

print("\nItens selecionados:")
itens = [
    ("Item 1", x1, 0.55, 250),
    ("Item 2", x2, 0.60, 300),
    ("Item 3", x3, 0.70, 500),
    ("Item 4", x4, 0.75, 700),
    ("Item 5", x5, 0.85, 750),
    ("Item 6", x6, 0.90, 900),
    ("Item 7", x7, 0.95, 950)
]

for nome, var, volume, valor in itens:
    if var.varValue == 1:
        print(f"- {nome}: Volume = {volume}m³, Valor = R${valor}")

Status: Optimal
Valor Total = 3300.0
Volume Utilizado = 3.45

Itens selecionados:
- Item 4: Volume = 0.75m³, Valor = R$700
- Item 5: Volume = 0.85m³, Valor = R$750
- Item 6: Volume = 0.9m³, Valor = R$900
- Item 7: Volume = 0.95m³, Valor = R$950


## 1.3.2


Neste problema, precisamos selecionar um conjunto de salva-vidas para cobrir todas as horas de operação da piscina (1pm às 9pm). Cada salva-vidas está disponível certo intervalo de tempo e cobra certo preço, nosso objetivo é minimizar o custo total, garantindo que pelo menos um salva-vidas esteja presente em cada intervalo de tempo.

Na modelagem do problema, nossas variáveis serão:

$$x_1 = \begin{cases} 1 & \text{se Joy é selecionada} \\ 0 & \text{caso contrário} \end{cases}$$
$$x_2 = \begin{cases} 1 & \text{se Dean é selecionado} \\ 0 & \text{caso contrário} \end{cases}$$
$$x_3 = \begin{cases} 1 & \text{se Tim é selecionado} \\ 0 & \text{caso contrário} \end{cases}$$
$$x_4 = \begin{cases} 1 & \text{se Celicia é selecionada} \\ 0 & \text{caso contrário} \end{cases}$$
$$x_5 = \begin{cases} 1 & \text{se Beth é selecionada} \\ 0 & \text{caso contrário} \end{cases}$$
$$x_6 = \begin{cases} 1 & \text{se Ivar é selecionado} \\ 0 & \text{caso contrário} \end{cases}$$
$$x_7 = \begin{cases} 1 & \text{se Eilene é selecionada} \\ 0 & \text{caso contrário} \end{cases}$$

Para todas elas, temos a restrição $x_i \in \{0, 1\}$.

As condições do problema são modeladas como:

1. **Cobertura das 13h-14h:** Joy ou Dean devem estar presentes
   $$x_1 + x_2 \geq 1$$

2. **Cobertura das 14h-15h:** Joy ou Dean devem estar presentes
   $$x_1 + x_2 \geq 1$$

3. **Cobertura das 15h-16h:** Joy deve estar presente
   $$x_1 \geq 1$$

4. **Cobertura das 16h-17h:** Joy, Tim ou Celicia devem estar presentes
   $$x_1 + x_3 + x_4 \geq 1$$

5. **Cobertura das 17h-18h:** Tim, Celicia ou Ivar devem estar presentes
   $$x_3 + x_4 + x_6 \geq 1$$

6. **Cobertura das 18h-19h:** Tim, Celicia, Beth ou Ivar devem estar presentes
   $$x_3 + x_4 + x_5 + x_6 \geq 1$$

7. **Cobertura das 19h-20h:** Celicia, Beth ou Ivar devem estar presentes
   $$x_4 + x_5 + x_6 \geq 1$$

8. **Cobertura das 20h-21h:** Celicia, Beth ou Eilene devem estar presentes
   $$x_4 + x_5 + x_7 \geq 1$$

A função objetivo é minimizar o custo total:

$$\min Z = 30x_1 + 18x_2 + 21x_3 + 38x_4 + 20x_5 + 22x_6 + 9x_7$$


In [25]:
prob = pl.LpProblem("Selecao_Salva_Vidas", pl.LpMinimize)

x1 = pl.LpVariable("x1", cat='Binary')
x2 = pl.LpVariable("x2", cat='Binary')
x3 = pl.LpVariable("x3", cat='Binary')
x4 = pl.LpVariable("x4", cat='Binary')
x5 = pl.LpVariable("x5", cat='Binary')
x6 = pl.LpVariable("x6", cat='Binary')
x7 = pl.LpVariable("x7", cat='Binary')

prob += 30*x1 + 18*x2 + 21*x3 + 38*x4 + 20*x5 + 22*x6 + 9*x7

prob += x1 + x2 >= 1
prob += x1 + x2 >= 1
prob += x1 >= 1
prob += x1 + x3 + x4 >= 1
prob += x3 + x4 + x6 >= 1
prob += x3 + x4 + x5 + x6 >= 1
prob += x4 + x5 + x6 >= 1
prob += x4 + x5 + x7 >= 1

prob.solve()

print("Status:", pl.LpStatus[prob.status])
print("Custo Total =", pl.value(prob.objective))
print("\nSalva-vidas selecionados:")
nomes = ["Joy", "Dean", "Tim", "Celicia", "Beth", "Ivar", "Eilene"]
variaveis = [x1, x2, x3, x4, x5, x6, x7]
for nome, var in zip(nomes, variaveis):
    print(var.varValue)
    if var.varValue == 1:
        print(f"- {nome}")

Status: Optimal
Custo Total = 61.0

Salva-vidas selecionados:
1.0
- Joy
0.0
0.0
0.0
0.0
1.0
- Ivar
1.0
- Eilene


## 1.3.3


Neste problema, queremos selecionar um subconjunto de itens para colocar na mala de modo a maximizar o valor total dos itens selecionados sem exceder o limite de peso de 20 kg. Cada item pode ser selecionado ou não.

Seja $x_i$ uma variável binária que indica se o item $i$ é selecionado ou não:

$$
x_i =
\begin{cases}
1 & \text{se o item } i \text{ é selecionado} \\
0 & \text{caso contrário}
\end{cases}
$$

para $i \in \{A, B, C, D, E, F\}$

**(a)** Para formular esse problema como um IP definimos o seguinte:

Nossa **função objetivo**, que busca maximizar o valor total dos itens selecionados

$$
\max Z = 60x_A + 70x_B + 40x_C + 70x_D + 16x_E + 100x_F
$$

Temos a restrição de que o peso total não pode exceder 20 kg:

$$
6x_A + 7x_B + 4x_C + 9x_D + 3x_E + 8x_F \leq 20
$$

Como já foi dito, devemos definir $x_i$ como uma variável binária, (ou inteira entre 0 e 1).

$$
x_i \in \{0,1\} \quad \forall i \in \{A,B,C,D,E,F\}
$$

**(b)** Aqui adiciona-se a restrição de que o Item D só pode ser selecionado se item A for selecionado.

Queremos que $x_D = 1$ apenas se $x_A = 1$. Isso é modelado pela restrição:

$$
x_D \leq x_A
$$

**(c)** Agora é permitido exceder o limite de peso com custo adicional

Seja $y \geq 0$ uma variável inteira que representa o número de quilogramas excedidos além do limite de 20 kg.

Nossa função objetivo muda, pois desejamos maximizar o valor total dos itens menos o custo do excesso de peso:

$$
\max Z = 60x_A + 70x_B + 40x_C + 70x_D + 16x_E + 100x_F - 15y
$$

Já na restrição de peso, fazemos com que o peso total possa exceder 20 kg, mas o excesso é capturado pela variável $y$:

$$
6x_A + 7x_B + 4x_C + 9x_D + 3x_E + 8x_F \leq 20 + y
$$

Temos então as seguintes variáveis:

$$
x_i \in \{0,1\} \quad \forall i \in \{A,B,C,D,E,F\}
$$

$$
y \geq 0, \quad y \in \mathbb{Z}
$$

Então, a formulação IP deste problema fica sendo:

$$
\max Z = 60x_A + 70x_B + 40x_C + 70x_D + 16x_E + 100x_F - 15y
$$

Sujeito a:

$$
6x_A + 7x_B + 4x_C + 9x_D + 3x_E + 8x_F \leq 20 + y
$$

$$
x_D \leq x_A
$$

$$
x_i \in \{0,1\} \quad \forall i \in \{A,B,C,D,E,F\}
$$

$$
y \geq 0, \quad y \in \mathbb{Z}
$$


In [26]:
# (a) Formulação IP original
prob_a = pl.LpProblem("Knapsack_Original", pl.LpMaximize)

x_A = pl.LpVariable("x_A", cat='Binary')
x_B = pl.LpVariable("x_B", cat='Binary')
x_C = pl.LpVariable("x_C", cat='Binary')
x_D = pl.LpVariable("x_D", cat='Binary')
x_E = pl.LpVariable("x_E", cat='Binary')
x_F = pl.LpVariable("x_F", cat='Binary')

prob_a += 60*x_A + 70*x_B + 40*x_C + 70*x_D + 16*x_E + 100*x_F, "Total_Value"

prob_a += 6*x_A + 7*x_B + 4*x_C + 9*x_D + 3*x_E + 8*x_F <= 20, "Weight_Constraint"

prob_a.solve()
print("=== Item (a) ===")
print("Status:", pl.LpStatus[prob_a.status])
for v in prob_a.variables():
    print(f"{v.name} = {v.varValue}")
print(f"Valor total = ${pl.value(prob_a.objective)}\n")

# (b) Com restrição adicional: D só se A for selecionado
prob_b = pl.LpProblem("Knapsack_With_Dependency", pl.LpMaximize)

x_A = pl.LpVariable("x_A", cat='Binary')
x_B = pl.LpVariable("x_B", cat='Binary')
x_C = pl.LpVariable("x_C", cat='Binary')
x_D = pl.LpVariable("x_D", cat='Binary')
x_E = pl.LpVariable("x_E", cat='Binary')
x_F = pl.LpVariable("x_F", cat='Binary')

prob_b += 60*x_A + 70*x_B + 40*x_C + 70*x_D + 16*x_E + 100*x_F, "Total_Value"

prob_b += 6*x_A + 7*x_B + 4*x_C + 9*x_D + 3*x_E + 8*x_F <= 20, "Weight_Constraint"
prob_b += x_D <= x_A, "D_requires_A"

prob_b.solve()
print("=== Item (b) ===")
print("Status:", pl.LpStatus[prob_b.status])
for v in prob_b.variables():
    print(f"{v.name} = {v.varValue}")
print(f"Valor total = ${pl.value(prob_b.objective)}\n")

# (c) Permitir excesso de peso com custo
prob_c = pl.LpProblem("Knapsack_With_Overage", pl.LpMaximize)

x_A = pl.LpVariable("x_A", cat='Binary')
x_B = pl.LpVariable("x_B", cat='Binary')
x_C = pl.LpVariable("x_C", cat='Binary')
x_D = pl.LpVariable("x_D", cat='Binary')
x_E = pl.LpVariable("x_E", cat='Binary')
x_F = pl.LpVariable("x_F", cat='Binary')
y = pl.LpVariable("y", lowBound=0, cat='Integer')  # kg excedidos

prob_c += 60*x_A + 70*x_B + 40*x_C + 70*x_D + 16*x_E + 100*x_F - 15*y, "Net_Value"

prob_c += 6*x_A + 7*x_B + 4*x_C + 9*x_D + 3*x_E + 8*x_F <= 20 + y, "Weight_With_Overage"
prob_c += x_D <= x_A, "D_requires_A"

prob_c.solve()
print("=== Item (c) ===")
print("Status:", pl.LpStatus[prob_c.status])
for v in prob_c.variables():
    print(f"{v.name} = {v.varValue}")

peso_total = (6*x_A.varValue + 7*x_B.varValue + 4*x_C.varValue +
              9*x_D.varValue + 3*x_E.varValue + 8*x_F.varValue)
print(f"Peso total = {peso_total} kg")
print(f"Valor líquido = ${pl.value(prob_c.objective)}")

=== Item (a) ===
Status: Optimal
x_A = 0.0
x_B = 1.0
x_C = 1.0
x_D = 0.0
x_E = 0.0
x_F = 1.0
Valor total = $210.0

=== Item (b) ===
Status: Optimal
x_A = 0.0
x_B = 1.0
x_C = 1.0
x_D = 0.0
x_E = 0.0
x_F = 1.0
Valor total = $210.0

=== Item (c) ===
Status: Optimal
x_A = 1.0
x_B = 1.0
x_C = 0.0
x_D = 0.0
x_E = 0.0
x_F = 1.0
y = 1.0
Peso total = 21.0 kg
Valor líquido = $215.0


## 1.3.4


Neste problema, queremos atribuir quartos a clientes de modo a maximizar a receita total do hotel. Cada quarto deve ser atribuído a exatamente um cliente, e cada cliente pode receber no máximo um quarto.

Seja $x_{ij}$ uma variável binária que indica se o cliente $i$ recebe o quarto $j$:

$$
x_{ij} =
\begin{cases}
1 & \text{se o cliente } i \text{ recebe o quarto } j \\
0 & \text{caso contrário}
\end{cases}
$$

onde $i \in \{\text{Abby}, \text{Bob}, \text{Clarence}, \text{Donald}\}$ e $j \in \{1, 2, 3\}$

**(a) Formulação IP original**

Nossa função objetivo que buscamos maximizar é a receita total:

$$
\max Z = 60x_{A1} + 50x_{A2} + 40x_{B1} + 70x_{B2} + 80x_{B3} + 55x_{C2} + 75x_{C3} + 65x_{D1} + 90x_{D2}
$$

Como cada quarto é atribuído a exatamente um cliente, adicionamos as restrições

$$
x_{A1} + x_{B1} + x_{C1} + x_{D1} = 1
$$

$$
x_{A2} + x_{B2} + x_{C2} + x_{D2} = 1
$$

$$
x_{A3} + x_{B3} + x_{C3} + x_{D3} = 1
$$

Cada cliente recebe no máximo um quarto:

$$
x_{A1} + x_{A2} + x_{A3} \leq 1
$$

$$
x_{B1} + x_{B2} + x_{B3} \leq 1
$$

$$
x_{C1} + x_{C2} + x_{C3} \leq 1
$$

$$
x_{D1} + x_{D2} + x_{D3} \leq 1
$$

Então, adicionamos as restrições de integralidade.

$$
x_{ij} \in \{0,1\} \quad \forall i,j
$$

**(b) Restrição adicional: Não alugar para Abby e Bob simultaneamente**

Queremos que Abby e Bob não sejam ambos selecionados:

$$
x_{A1} + x_{A2} + x_{A3} + x_{B1} + x_{B2} + x_{B3} \leq 1
$$

**Formulação IP completa para (b):**

$$
\max Z = 60x_{A1} + 50x_{A2} + 40x_{B1} + 70x_{B2} + 80x_{B3} + 55x_{C2} + 75x_{C3} + 65x_{D1} + 90x_{D2}
$$

Sujeito a:

$$
x_{A1} + x_{B1} + x_{C1} + x_{D1} = 1
$$

$$
x_{A2} + x_{B2} + x_{C2} + x_{D2} = 1
$$

$$
x_{A3} + x_{B3} + x_{C3} + x_{D3} = 1
$$

$$
x_{A1} + x_{A2} + x_{A3} \leq 1
$$

$$
x_{B1} + x_{B2} + x_{B3} \leq 1
$$

$$
x_{C1} + x_{C2} + x_{C3} \leq 1
$$

$$
x_{D1} + x_{D2} + x_{D3} \leq 1
$$

$$
x_{A1} + x_{A2} + x_{A3} + x_{B1} + x_{B2} + x_{B3} \leq 1
$$

$$
x_{ij} \in \{0,1\} \quad \forall i,j
$$


In [27]:
# (a) Formulação IP original
prob_a = pl.LpProblem("Hotel_Assignment_Original", pl.LpMaximize)

# Variáveis de decisão
x_A1 = pl.LpVariable("x_A1", cat='Binary')
x_A2 = pl.LpVariable("x_A2", cat='Binary')
x_A3 = pl.LpVariable("x_A3", cat='Binary')
x_B1 = pl.LpVariable("x_B1", cat='Binary')
x_B2 = pl.LpVariable("x_B2", cat='Binary')
x_B3 = pl.LpVariable("x_B3", cat='Binary')
x_C1 = pl.LpVariable("x_C1", cat='Binary')
x_C2 = pl.LpVariable("x_C2", cat='Binary')
x_C3 = pl.LpVariable("x_C3", cat='Binary')
x_D1 = pl.LpVariable("x_D1", cat='Binary')
x_D2 = pl.LpVariable("x_D2", cat='Binary')
x_D3 = pl.LpVariable("x_D3", cat='Binary')

# Função objetivo
prob_a += (60*x_A1 + 50*x_A2 + 40*x_B1 + 70*x_B2 + 80*x_B3 +
          55*x_C2 + 75*x_C3 + 65*x_D1 + 90*x_D2, "Total_Revenue")

# Restrições - Cada quarto para exatamente um cliente
prob_a += x_A1 + x_B1 + x_C1 + x_D1 == 1, "Room1_Assignment"
prob_a += x_A2 + x_B2 + x_C2 + x_D2 == 1, "Room2_Assignment"
prob_a += x_A3 + x_B3 + x_C3 + x_D3 == 1, "Room3_Assignment"

# Restrições - Cada cliente no máximo um quarto
prob_a += x_A1 + x_A2 + x_A3 <= 1, "Abby_MaxOne"
prob_a += x_B1 + x_B2 + x_B3 <= 1, "Bob_MaxOne"
prob_a += x_C1 + x_C2 + x_C3 <= 1, "Clarence_MaxOne"
prob_a += x_D1 + x_D2 + x_D3 <= 1, "Donald_MaxOne"

# Resolver
prob_a.solve()
print("=== Item (a) ===")
print("Status:", pl.LpStatus[prob_a.status])
for v in prob_a.variables():
    if v.varValue > 0:
        print(f"{v.name} = {v.varValue}")
print(f"Receita total = ${pl.value(prob_a.objective)}\n")

# (b) Com restrição adicional: Não alugar para Abby e Bob simultaneamente
prob_b = pl.LpProblem("Hotel_Assignment_No_Abby_Bob", pl.LpMaximize)

# Variáveis de decisão
x_A1 = pl.LpVariable("x_A1", cat='Binary')
x_A2 = pl.LpVariable("x_A2", cat='Binary')
x_A3 = pl.LpVariable("x_A3", cat='Binary')
x_B1 = pl.LpVariable("x_B1", cat='Binary')
x_B2 = pl.LpVariable("x_B2", cat='Binary')
x_B3 = pl.LpVariable("x_B3", cat='Binary')
x_C1 = pl.LpVariable("x_C1", cat='Binary')
x_C2 = pl.LpVariable("x_C2", cat='Binary')
x_C3 = pl.LpVariable("x_C3", cat='Binary')
x_D1 = pl.LpVariable("x_D1", cat='Binary')
x_D2 = pl.LpVariable("x_D2", cat='Binary')
x_D3 = pl.LpVariable("x_D3", cat='Binary')

# Função objetivo
prob_b += (60*x_A1 + 50*x_A2 + 40*x_B1 + 70*x_B2 + 80*x_B3 +
          55*x_C2 + 75*x_C3 + 65*x_D1 + 90*x_D2, "Total_Revenue")

# Restrições - Cada quarto para exatamente um cliente
prob_b += x_A1 + x_B1 + x_C1 + x_D1 == 1, "Room1_Assignment"
prob_b += x_A2 + x_B2 + x_C2 + x_D2 == 1, "Room2_Assignment"
prob_b += x_A3 + x_B3 + x_C3 + x_D3 == 1, "Room3_Assignment"

# Restrições - Cada cliente no máximo um quarto
prob_b += x_A1 + x_A2 + x_A3 <= 1, "Abby_MaxOne"
prob_b += x_B1 + x_B2 + x_B3 <= 1, "Bob_MaxOne"
prob_b += x_C1 + x_C2 + x_C3 <= 1, "Clarence_MaxOne"
prob_b += x_D1 + x_D2 + x_D3 <= 1, "Donald_MaxOne"

# Restrição adicional: Não alugar para Abby e Bob simultaneamente
prob_b += x_A1 + x_A2 + x_A3 + x_B1 + x_B2 + x_B3 <= 1, "No_Abby_Bob"

prob_b.solve()
print("=== Item (b) ===")
print("Status:", pl.LpStatus[prob_b.status])
for v in prob_b.variables():
    if v.varValue > 0:
        print(f"{v.name} = {v.varValue}")
print(f"Receita total = ${pl.value(prob_b.objective)}")

=== Item (a) ===
Status: Optimal
x_A1 = 1.0
x_B3 = 1.0
x_D2 = 1.0
Receita total = $230.0

=== Item (b) ===
Status: Optimal
x_A1 = 1.0
x_C3 = 1.0
x_D2 = 1.0
Receita total = $225.0


## 1.3.5


Neste problema, queremos maximizar o valor total das caixas transportadas em um avião, respeitando os limites de peso e volume em cada segmento (Front, Middle, Back), as disponibilidades de cada tipo de caixa e as restrições de balanceamento do avião.

---

**Variáveis de decisão:**
Seja $x_{ij}$ o número de caixas do tipo $i$ alocadas no segmento $j$:

$$
x_{ij} \in \mathbb{Z}_{\geq 0}
$$

onde $i \in \{A, B, C, D, E\}$ e $j \in \{\text{Front}, \text{Middle}, \text{Back}\}$

---

**Disponibilidade de caixas:**

- Tipo A: 12 caixas
- Tipo B: 8 caixas
- Tipo C: 22 caixas
- Tipo D: 15 caixas
- Tipo E: 11 caixas

---

**Função objetivo:**
Maximizar o valor total das caixas transportadas:

$$
\max Z = 50000(x_{A,F} + x_{A,M} + x_{A,B}) + 60000(x_{B,F} + x_{B,M} + x_{B,B}) + 90000(x_{C,F} + x_{C,M} + x_{C,B}) + 40000(x_{D,F} + x_{D,M} + x_{D,B}) + 30000(x_{E,F} + x_{E,M} + x_{E,B})
$$

---

**Restrições de disponibilidade:**

**Cada tipo de caixa não pode exceder a quantidade disponível:**

$$
x_{A,F} + x_{A,M} + x_{A,B} \leq 12
$$

$$
x_{B,F} + x_{B,M} + x_{B,B} \leq 8
$$

$$
x_{C,F} + x_{C,M} + x_{C,B} \leq 22
$$

$$
x_{D,F} + x_{D,M} + x_{D,B} \leq 15
$$

$$
x_{E,F} + x_{E,M} + x_{E,B} \leq 11
$$

---

**Restrições de volume por segmento:**

**Front (200 m³):**

$$
25x_{A,F} + 15x_{B,F} + 13x_{C,F} + 20x_{D,F} + 16x_{E,F} \leq 200
$$

**Middle (500 m³):**

$$
25x_{A,M} + 15x_{B,M} + 13x_{C,M} + 20x_{D,M} + 16x_{E,M} \leq 500
$$

**Back (300 m³):**

$$
25x_{A,B} + 15x_{B,B} + 13x_{C,B} + 20x_{D,B} + 16x_{E,B} \leq 300
$$

---

**Restrições de peso por segmento:**

**Front (8000 kg):**

$$
500x_{A,F} + 1500x_{B,F} + 2100x_{C,F} + 600x_{D,F} + 400x_{E,F} \leq 8000
$$

**Middle (20000 kg):**

$$
500x_{A,M} + 1500x_{B,M} + 2100x_{C,M} + 600x_{D,M} + 400x_{E,M} \leq 20000
$$

**Back (6000 kg):**

$$
500x_{A,B} + 1500x_{B,B} + 2100x_{C,B} + 600x_{D,B} + 400x_{E,B} \leq 6000
$$

---

**Restrições de balanceamento:**

**Peso do Middle ≥ Peso do Front + Peso do Back:**

$$
500x_{A,M} + 1500x_{B,M} + 2100x_{C,M} + 600x_{D,M} + 400x_{E,M} \geq 500x_{A,F} + 1500x_{B,F} + 2100x_{C,F} + 600x_{D,F} + 400x_{E,F} + 500x_{A,B} + 1500x_{B,B} + 2100x_{C,B} + 600x_{D,B} + 400x_{E,B}
$$

**Peso do Middle ≤ 2 × (Peso do Front + Peso do Back):**

$$
500x_{A,M} + 1500x_{B,M} + 2100x_{C,M} + 600x_{D,M} + 400x_{E,M} \leq 2 \times (500x_{A,F} + 1500x_{B,F} + 2100x_{C,F} + 600x_{D,F} + 400x_{E,F} + 500x_{A,B} + 1500x_{B,B} + 2100x_{C,B} + 600x_{D,B} + 400x_{E,B})
$$

---

**Restrições de integralidade e não-negatividade:**

$$
x_{ij} \geq 0, \quad x_{ij} \in \mathbb{Z} \quad \forall i,j
$$

---

**Formulação IP completa:**

$$
\max Z = 50000\sum_{j} x_{A,j} + 60000\sum_{j} x_{B,j} + 90000\sum_{j} x_{C,j} + 40000\sum_{j} x_{D,j} + 30000\sum_{j} x_{E,j}
$$

Sujeito a:

$$
\sum_{j} x_{A,j} \leq 12, \quad \sum_{j} x_{B,j} \leq 8, \quad \sum_{j} x_{C,j} \leq 22, \quad \sum_{j} x_{D,j} \leq 15, \quad \sum_{j} x_{E,j} \leq 11
$$

$$
25x_{A,F} + 15x_{B,F} + 13x_{C,F} + 20x_{D,F} + 16x_{E,F} \leq 200
$$

$$
25x_{A,M} + 15x_{B,M} + 13x_{C,M} + 20x_{D,M} + 16x_{E,M} \leq 500
$$

$$
25x_{A,B} + 15x_{B,B} + 13x_{C,B} + 20x_{D,B} + 16x_{E,B} \leq 300
$$

$$
500x_{A,F} + 1500x_{B,F} + 2100x_{C,F} + 600x_{D,F} + 400x_{E,F} \leq 8000
$$

$$
500x_{A,M} + 1500x_{B,M} + 2100x_{C,M} + 600x_{D,M} + 400x_{E,M} \leq 20000
$$

$$
500x_{A,B} + 1500x_{B,B} + 2100x_{C,B} + 600x_{D,B} + 400x_{E,B} \leq 6000
$$

$$
\text{Peso}_{M} \geq \text{Peso}_{F} + \text{Peso}_{B}
$$

$$
\text{Peso}_{M} \leq 2 \times (\text{Peso}_{F} + \text{Peso}_{B})
$$

$$
x_{ij} \geq 0, \quad x_{ij} \in \mathbb{Z} \quad \forall i,j
$$


In [28]:
prob = pl.LpProblem("Cargo_Plane_Optimization", pl.LpMaximize)

# Variáveis de decisão
crate_types = ['A', 'B', 'C', 'D', 'E']
segments = ['Front', 'Middle', 'Back']

x = pl.LpVariable.dicts("x",
                         [(i, j) for i in crate_types for j in segments],
                         lowBound=0, cat='Integer')

# Parâmetros
availability = {'A': 12, 'B': 8, 'C': 22, 'D': 15, 'E': 11}
weight = {'A': 500, 'B': 1500, 'C': 2100, 'D': 600, 'E': 400}
volume = {'A': 25, 'B': 15, 'C': 13, 'D': 20, 'E': 16}
value = {'A': 50000, 'B': 60000, 'C': 90000, 'D': 40000, 'E': 30000}

volume_capacity = {'Front': 200, 'Middle': 500, 'Back': 300}
weight_capacity = {'Front': 8000, 'Middle': 20000, 'Back': 6000}

# Função objetivo
prob += pl.lpSum(value[i] * x[(i, j)] for i in crate_types for j in segments)

# Restrições de disponibilidade
for i in crate_types:
    prob += pl.lpSum(x[(i, j)] for j in segments) <= availability[i]

# Restrições de volume
for j in segments:
    prob += pl.lpSum(volume[i] * x[(i, j)] for i in crate_types) <= volume_capacity[j]

# Restrições de peso
for j in segments:
    prob += pl.lpSum(weight[i] * x[(i, j)] for i in crate_types) <= weight_capacity[j]

# Restrições de balanceamento
weight_front = pl.lpSum(weight[i] * x[(i, 'Front')] for i in crate_types)
weight_middle = pl.lpSum(weight[i] * x[(i, 'Middle')] for i in crate_types)
weight_back = pl.lpSum(weight[i] * x[(i, 'Back')] for i in crate_types)

prob += weight_middle >= weight_front + weight_back
prob += weight_middle <= 2 * (weight_front + weight_back)

# Resolver
prob.solve()

print("=== Problema 5 - Otimização de Carga no Avião ===")
print("Status:", pl.LpStatus[prob.status])
print(f"Valor total = ${pl.value(prob.objective):,.0f}\n")

# Mostrar resultados por segmento
for j in segments:
    print(f"--- Segmento {j} ---")
    total_weight = 0
    total_volume = 0
    for i in crate_types:
        if x[(i, j)].varValue > 0:
            print(f"  {i}: {x[(i, j)].varValue} caixas")
            total_weight += weight[i] * x[(i, j)].varValue
            total_volume += volume[i] * x[(i, j)].varValue
    print(f"  Peso total: {total_weight} kg (capacidade: {weight_capacity[j]} kg)")
    print(f"  Volume total: {total_volume} m³ (capacidade: {volume_capacity[j]} m³)\n")

# Mostrar utilização total por tipo de caixa
print("--- Utilização Total por Tipo ---")
for i in crate_types:
    total_used = sum(x[(i, j)].varValue for j in segments)
    print(f"  {i}: {total_used}/{availability[i]} caixas utilizadas")

# Verificar restrições de balanceamento
print(f"\n--- Balanceamento ---")
print(f"Peso Front: {weight_front.value()} kg")
print(f"Peso Middle: {weight_middle.value()} kg")
print(f"Peso Back: {weight_back.value()} kg")
print(f"Middle ≥ Front + Back: {weight_middle.value()} ≥ {weight_front.value() + weight_back.value()} = {weight_middle.value() >= weight_front.value() + weight_back.value()}")
print(f"Middle ≤ 2×(Front + Back): {weight_middle.value()} ≤ {2 * (weight_front.value() + weight_back.value())} = {weight_middle.value() <= 2 * (weight_front.value() + weight_back.value())}")

=== Problema 5 - Otimização de Carga no Avião ===
Status: Optimal
Valor total = $2,130,000

--- Segmento Front ---
  A: 1.0 caixas
  C: 3.0 caixas
  D: 2.0 caixas
  Peso total: 8000.0 kg (capacidade: 8000 kg)
  Volume total: 104.0 m³ (capacidade: 200 m³)

--- Segmento Middle ---
  B: 4.0 caixas
  C: 1.0 caixas
  D: 13.0 caixas
  E: 10.0 caixas
  Peso total: 19900.0 kg (capacidade: 20000 kg)
  Volume total: 493.0 m³ (capacidade: 500 m³)

--- Segmento Back ---
  A: 11.0 caixas
  E: 1.0 caixas
  Peso total: 5900.0 kg (capacidade: 6000 kg)
  Volume total: 291.0 m³ (capacidade: 300 m³)

--- Utilização Total por Tipo ---
  A: 12.0/12 caixas utilizadas
  B: 4.0/8 caixas utilizadas
  C: 4.0/22 caixas utilizadas
  D: 15.0/15 caixas utilizadas
  E: 11.0/11 caixas utilizadas

--- Balanceamento ---
Peso Front: 8000.0 kg
Peso Middle: 19900.0 kg
Peso Back: 5900.0 kg
Middle ≥ Front + Back: 19900.0 ≥ 13900.0 = True
Middle ≤ 2×(Front + Back): 19900.0 ≤ 27800.0 = True


## 1.3.7


Neste problema, a empresa C & O opera um oleoduto que bombeia óleo de Alberta para três estados (A, B, C) nos EUA. Existem quatro linhas de entrada, sendo que a linha do Yukon tem um custo fixo de ativação. O objetivo é minimizar o custo diário total atendendo exatamente à demanda de cada estado.

---

**Variáveis de decisão:**

Seja $y_i$ o fluxo de óleo (barris/dia) da linha de entrada $i$:

- $y_1$: fluxo da Input 1
- $y_2$: fluxo da Input 2
- $y_3$: fluxo da Input 3
- $y_Y$: fluxo da Yukon

Seja $z$ uma variável binária que indica se a linha do Yukon é ativada:

$$
z =
\begin{cases}
1 & \text{se a linha Yukon é usada} \\
0 & \text{caso contrário}
\end{cases}
$$

---

**Capacidades das linhas de entrada:**

- Input 1: 4000 barris/dia
- Input 2: 2000 barris/dia
- Input 3: 3000 barris/dia
- Yukon: 10000 barris/dia

---

**Custos variáveis:**

- Input 1: \$70 por barril
- Input 2: \$50 por barril
- Input 3: \$30 por barril
- Yukon: \$60 por barril

**Custo fixo:** Yukon: \$11000 por dia

---

**Demanda dos estados:**

- Estado A: 3500 barris/dia
- Estado B: 3000 barris/dia
- Estado C: 4000 barris/dia

---

**Função objetivo:**
Minimizar o custo total diário (custos variáveis + custo fixo do Yukon):

$$
\min Z = 70y_1 + 50y_2 + 30y_3 + 60y_Y + 11000z
$$

---

**Restrições de capacidade:**

**Cada linha de entrada não pode exceder sua capacidade:**

$$
y_1 \leq 4000
$$

$$
y_2 \leq 2000
$$

$$
y_3 \leq 3000
$$

$$
y_Y \leq 10000z
$$

---

**Restrições de demanda:**

**Estado A (pode receber apenas de Input 1 ou Yukon):**

$$
y_1 + y_Y = 3500
$$

**Estado B (pode receber apenas de Input 2):**

$$
y_2 = 3000
$$

**Estado C (pode receber apenas de Input 3):**

$$
y_3 = 4000
$$

---

**Restrições de não-negatividade e integralidade:**

$$
y_1, y_2, y_3, y_Y \geq 0
$$

$$
z \in \{0, 1\}
$$

---

**Formulação IP completa:**

$$
\min Z = 70y_1 + 50y_2 + 30y_3 + 60y_Y + 11000z
$$

Sujeito a:

$$
y_1 \leq 4000
$$

$$
y_2 \leq 2000
$$

$$
y_3 \leq 3000
$$

$$
y_Y \leq 10000z
$$

$$
y_1 + y_Y = 3500
$$

$$
y_2 = 3000
$$

$$
y_3 = 4000
$$

$$
y_1, y_2, y_3, y_Y \geq 0
$$

$$
z \in \{0, 1\}
$$


In [29]:
# Criar o problema
prob = pl.LpProblem("Oil_Pipeline_Optimization", pl.LpMinimize)

# Variáveis de decisão
y1 = pl.LpVariable("y1", lowBound=0, upBound=4000, cat='Continuous')  # Input 1
y2 = pl.LpVariable("y2", lowBound=0, upBound=2000, cat='Continuous')  # Input 2
y3 = pl.LpVariable("y3", lowBound=0, upBound=3000, cat='Continuous')  # Input 3
yY = pl.LpVariable("yY", lowBound=0, cat='Continuous')  # Yukon
z = pl.LpVariable("z", cat='Binary')  # Ativação Yukon

# Função objetivo
prob += 70*y1 + 50*y2 + 30*y3 + 60*yY + 11000*z

# Restrições
prob += yY <= 10000*z  # Yukon só pode ser usado se ativado
prob += y1 + yY == 3500  # Demanda Estado A
prob += y2 == 3000  # Demanda Estado B
prob += y3 == 4000  # Demanda Estado C

# Resolver
prob.solve()

print("=== Problema 7 - Otimização do Oleoduto ===")
print("Status:", pl.LpStatus[prob.status])
print(f"Custo total diário = ${pl.value(prob.objective):,.0f}\n")

print("--- Fluxo Ótimo ---")
print(f"Input 1: {y1.varValue} barris/dia")
print(f"Input 2: {y2.varValue} barris/dia")
print(f"Input 3: {y3.varValue} barris/dia")
print(f"Yukon: {yY.varValue} barris/dia")
print(f"Yukon ativado: {z.varValue}\n")

print("--- Verificação Demanda ---")
print(f"Estado A: {y1.varValue + yY.varValue} barris/dia (demanda: 3500)")
print(f"Estado B: {y2.varValue} barris/dia (demanda: 3000)")
print(f"Estado C: {y3.varValue} barris/dia (demanda: 4000)\n")

print("--- Custos ---")
print(f"Custo Input 1: ${70 * y1.varValue:,.0f}")
print(f"Custo Input 2: ${50 * y2.varValue:,.0f}")
print(f"Custo Input 3: ${30 * y3.varValue:,.0f}")
print(f"Custo Yukon: ${60 * yY.varValue:,.0f}")
print(f"Custo fixo Yukon: ${11000 * z.varValue:,.0f}")

=== Problema 7 - Otimização do Oleoduto ===
Status: Infeasible
Custo total diário = $480,000

--- Fluxo Ótimo ---
Input 1: 0.0 barris/dia
Input 2: 3000.0 barris/dia
Input 3: 4000.0 barris/dia
Yukon: 3500.0 barris/dia
Yukon ativado: 0.0

--- Verificação Demanda ---
Estado A: 3500.0 barris/dia (demanda: 3500)
Estado B: 3000.0 barris/dia (demanda: 3000)
Estado C: 4000.0 barris/dia (demanda: 4000)

--- Custos ---
Custo Input 1: $0
Custo Input 2: $150,000
Custo Input 3: $120,000
Custo Yukon: $210,000
Custo fixo Yukon: $0


## 1.3.10


Neste problema, a empresa KW mining tem uma mina a céu aberto com 12 blocos de minério dispostos em camadas. A condição de mineirabilidade estabelece que nenhum bloco pode ser minerado sem que todos os blocos que estão acima dele (parcial ou totalmente) tenham sido minerados.

---

**Estrutura dos blocos:**

- Camada 1: Blocos 1, 2, 3, 4, 5
- Camada 2: Blocos 6, 7, 8, 9
- Camada 3: Blocos 10, 11, 12

---

**Relações de precedência:**

- Bloco 6 requer Bloco 1
- Bloco 7 requer Blocos 2 e 3
- Bloco 8 requer Blocos 3 e 4
- Bloco 9 requer Blocos 4 e 5
- Bloco 10 requer Bloco 6
- Bloco 11 requer Blocos 6 e 7
- Bloco 12 requer Blocos 8 e 9

---

**(a) Formulação IP básica**

**Variáveis de decisão:**
Seja $x_i$ uma variável binária que indica se o bloco $i$ é minerado:

$$
x_i =
\begin{cases}
1 & \text{se o bloco } i \text{ é minerado} \\
0 & \text{caso contrário}
\end{cases}
\quad \text{para } i = 1,\ldots,12
$$

**Função objetivo:**
Maximizar o valor total dos blocos minerados:

$$
\max Z = \sum_{i=1}^{12} m_i x_i
$$

**Restrições de precedência:**

$$
x_6 \leq x_1
$$

$$
x_7 \leq x_2,\quad x_7 \leq x_3
$$

$$
x_8 \leq x_3,\quad x_8 \leq x_4
$$

$$
x_9 \leq x_4,\quad x_9 \leq x_5
$$

$$
x_{10} \leq x_6
$$

$$
x_{11} \leq x_6,\quad x_{11} \leq x_7
$$

$$
x_{12} \leq x_8,\quad x_{12} \leq x_9
$$

**Restrição de no máximo 7 blocos:**

$$
\sum_{i=1}^{12} x_i \leq 7
$$

**Restrição de integralidade:**

$$
x_i \in \{0,1\} \quad \forall i = 1,\ldots,12
$$

---

**(b) Restrição adicional de volume**

**Variável adicional:** Seja $v_i$ o volume do bloco $i$

**Restrição de volume máximo:**

$$
\sum_{i=1}^{12} v_i x_i \leq 10000
$$

---

**(c) Condição de retorno de curto prazo**

**Variáveis adicionais:**
Seja $y_{it}$ uma variável binária que indica se o bloco $i$ é minerado no ano $t$:

$$
y_{it} =
\begin{cases}
1 & \text{se o bloco } i \text{ é minerado no ano } t \\
0 & \text{caso contrário}
\end{cases}
\quad \text{para } t = 1,2
$$

**Restrições adicionais:**

**Cada bloco é minerado no máximo uma vez:**

$$
y_{i1} + y_{i2} = x_i \quad \forall i = 1,\ldots,12
$$

**No máximo 2 blocos por ano:**

$$
\sum_{i=1}^{12} y_{i1} \leq 2,\quad \sum_{i=1}^{12} y_{i2} \leq 2
$$

**Precedência temporal - bloco só pode ser minerado no ano $t$ se todos acima foram minerados até ano $t-1$:**

$$
y_{i2} \leq x_j \quad \text{para todos } j \text{ acima de } i
$$

**Valor mínimo nos primeiros 2 anos:**

$$
\sum_{i=1}^{12} m_i (y_{i1} + y_{i2}) \geq 1000000
$$

**Formulação IP completa para (c):**

$$
\max Z = \sum_{i=1}^{12} m_i x_i
$$

Sujeito a:

$$
x_6 \leq x_1,\quad x_7 \leq x_2,\quad x_7 \leq x_3,\quad x_8 \leq x_3,\quad x_8 \leq x_4,\quad x_9 \leq x_4,\quad x_9 \leq x_5
$$

$$
x_{10} \leq x_6,\quad x_{11} \leq x_6,\quad x_{11} \leq x_7,\quad x_{12} \leq x_8,\quad x_{12} \leq x_9
$$

$$
\sum_{i=1}^{12} x_i \leq 7
$$

$$
y_{i1} + y_{i2} = x_i \quad \forall i = 1,\ldots,12
$$

$$
\sum_{i=1}^{12} y_{i1} \leq 2,\quad \sum_{i=1}^{12} y_{i2} \leq 2
$$

$$
y_{i2} \leq x_j \quad \text{para todos } j \text{ acima de } i
$$

$$
\sum_{i=1}^{12} m_i (y_{i1} + y_{i2}) \geq 1000000
$$

$$
x_i, y_{i1}, y_{i2} \in \{0,1\} \quad \forall i = 1,\ldots,12
$$


In [30]:
# Parâmetros de exemplo (valores fictícios)
m = [500000, 600000, 700000, 800000, 900000,
     400000, 500000, 600000, 700000, 300000, 400000, 500000]  # Valores m_i
v = [2000, 2500, 3000, 3500, 4000,
     1500, 2000, 2500, 3000, 1000, 1500, 2000]  # Volumes v_i

# (a) Formulação IP básica
prob_a = pl.LpProblem("Mining_Basic", pl.LpMaximize)

# Variáveis
x = pl.LpVariable.dicts("x", range(1, 13), cat='Binary')

# Função objetivo
prob_a += pl.lpSum(m[i-1] * x[i] for i in range(1, 13))

# Restrições de precedência
prob_a += x[6] <= x[1]
prob_a += x[7] <= x[2]
prob_a += x[7] <= x[3]
prob_a += x[8] <= x[3]
prob_a += x[8] <= x[4]
prob_a += x[9] <= x[4]
prob_a += x[9] <= x[5]
prob_a += x[10] <= x[6]
prob_a += x[11] <= x[6]
prob_a += x[11] <= x[7]
prob_a += x[12] <= x[8]
prob_a += x[12] <= x[9]

# No máximo 7 blocos
prob_a += pl.lpSum(x[i] for i in range(1, 13)) <= 7

# Resolver
prob_a.solve()
print("(a) Formulação Básica:")
print(f"Valor total = ${pl.value(prob_a.objective):,.0f}")
print("Blocos minerados:", [i for i in range(1, 13) if x[i].varValue > 0])
print()

# (b) Com restrição de volume
prob_b = pl.LpProblem("Mining_Volume", pl.LpMaximize)

# Variáveis
x = pl.LpVariable.dicts("x", range(1, 13), cat='Binary')

# Função objetivo e restrições de precedência (mesmas do (a))
prob_b += pl.lpSum(m[i-1] * x[i] for i in range(1, 13))
prob_b += x[6] <= x[1]
prob_b += x[7] <= x[2]
prob_b += x[7] <= x[3]
prob_b += x[8] <= x[3]
prob_b += x[8] <= x[4]
prob_b += x[9] <= x[4]
prob_b += x[9] <= x[5]
prob_b += x[10] <= x[6]
prob_b += x[11] <= x[6]
prob_b += x[11] <= x[7]
prob_b += x[12] <= x[8]
prob_b += x[12] <= x[9]
prob_b += pl.lpSum(x[i] for i in range(1, 13)) <= 7

# Restrição de volume
prob_b += pl.lpSum(v[i-1] * x[i] for i in range(1, 13)) <= 10000

prob_b.solve()
print("(b) Com Restrição de Volume:")
print(f"Valor total = ${pl.value(prob_b.objective):,.0f}")
print(f"Volume total = {sum(v[i-1] * x[i+1].varValue for i in range(12))} m³")
print("Blocos minerados:", [i for i in range(1, 13) if x[i].varValue > 0])

(a) Formulação Básica:
Valor total = $4,800,000
Blocos minerados: [2, 3, 4, 5, 7, 8, 9]

(b) Com Restrição de Volume:
Valor total = $2,500,000
Volume total = 13500.0 m³
Blocos minerados: [1, 2, 3, 6, 10]


## 1.3.11


Neste problema, queremos colocar N rainhas em um tabuleiro de xadrez $N \times N$ de forma que nenhuma duas rainhas compartilhem a mesma linha, coluna ou diagonal. Este é o conhecido problema das $N$-rainhas.

---

**Variáveis de decisão:**
Seja $x_{ij}$ uma variável binária que indica se uma rainha é colocada na posição $(i,j)$ do tabuleiro:

$$
x_{ij} =
\begin{cases}
1 & \text{se há uma rainha na posição } (i,j) \\
0 & \text{caso contrário}
\end{cases}
\quad \text{para } i,j = 1,\ldots,N
$$

---

**Restrições:**

**Exatamente uma rainha por linha:**

$$
\sum_{j=1}^{N} x_{ij} = 1 \quad \forall i = 1,\ldots,N
$$

**Exatamente uma rainha por coluna:**

$$
\sum_{i=1}^{N} x_{ij} = 1 \quad \forall j = 1,\ldots,N
$$

**Restrições de diagonal principal (\):**
Para cada diagonal $k = 2,\ldots,2N$:

$$
\sum_{i+j=k} x_{ij} \leq 1
$$

**Restrições de diagonal secundária (/):**
Para cada diagonal $k = -(N-1),\ldots,(N-1)$:

$$
\sum_{i-j=k} x_{ij} \leq 1
$$

---

**Formulação IP completa:**

Encontrar $x_{ij}$ tal que:

$$
\sum_{j=1}^{N} x_{ij} = 1 \quad \forall i = 1,\ldots,N
$$

$$
\sum_{i=1}^{N} x_{ij} = 1 \quad \forall j = 1,\ldots,N
$$

$$
\sum_{i+j=k} x_{ij} \leq 1 \quad \forall k = 2,\ldots,2N
$$

$$
\sum_{i-j=k} x_{ij} \leq 1 \quad \forall k = -(N-1),\ldots,(N-1)
$$

$$
x_{ij} \in \{0,1\} \quad \forall i,j = 1,\ldots,N
$$


In [31]:
for N in [4, 8]:
    print(f"\n--- N = {N} ---")

    # Todo o código do solver aqui (igual ao acima)
    prob = pl.LpProblem("N_Queens", pl.LpMinimize)
    x = pl.LpVariable.dicts("x", [(i, j) for i in range(N) for j in range(N)], cat='Binary')
    prob += 0

    # Restrições...
    for i in range(N):
        prob += pl.lpSum(x[(i, j)] for j in range(N)) == 1
    for j in range(N):
        prob += pl.lpSum(x[(i, j)] for i in range(N)) == 1
    for d in range(2, 2*N):
        cells_in_diag = [(i, j) for i in range(N) for j in range(N) if i + j + 2 == d]
        if cells_in_diag:
            prob += pl.lpSum(x[cell] for cell in cells_in_diag) <= 1
    for d in range(-N+1, N):
        cells_in_diag = [(i, j) for i in range(N) for j in range(N) if i - j == d]
        if cells_in_diag:
            prob += pl.lpSum(x[cell] for cell in cells_in_diag) <= 1

    prob.solve()

    if pl.LpStatus[prob.status] == 'Optimal':
        print(f"Solução encontrada para N={N}!")
        board = [['.' for _ in range(N)] for _ in range(N)]
        for i in range(N):
            for j in range(N):
                if x[(i, j)].varValue > 0.5:
                    board[i][j] = 'Q'
        print("\nTabuleiro:")
        for row in board:
            print(' '.join(row))
    else:
        print(f"Nenhuma solução encontrada para N={N}")


--- N = 4 ---
Solução encontrada para N=4!

Tabuleiro:
. . Q .
Q . . .
. . . Q
. Q . .

--- N = 8 ---
Solução encontrada para N=8!

Tabuleiro:
. . Q . . . . .
Q . . . . . . .
. . . . . . Q .
. . . . Q . . .
. . . . . . . Q
. Q . . . . . .
. . . Q . . . .
. . . . . Q . .


## 1.3.12


Neste problema, queremos verificar se existe uma solução para um jogo de Sudoku $9 \times 9$. O tabuleiro é dividido em $9$ submatrizes $3 \times 3$. A solução deve atribuir números de $1$ a $9$ a cada célula de forma que cada linha, cada coluna e cada submatriz $3 \times 3$ contenha cada número exatamente uma vez.

---

**Variáveis de decisão:**
Seja $x_{ijk}$ uma variável binária que indica se a célula $(i,j)$ contém o valor $k$:

$$
x_{ijk} =
\begin{cases}
1 & \text{se a célula } (i,j) \text{ contém o valor } k \\
0 & \text{caso contrário}
\end{cases}
\quad \text{para } i,j,k = 1,\ldots,9
$$

---

**Restrições:**

**Cada célula contém exatamente um valor:**

$$
\sum_{k=1}^{9} x_{ijk} = 1 \quad \forall i,j = 1,\ldots,9
$$

**Cada número aparece exatamente uma vez por linha:**

$$
\sum_{j=1}^{9} x_{ijk} = 1 \quad \forall i,k = 1,\ldots,9
$$

**Cada número aparece exatamente uma vez por coluna:**

$$
\sum_{i=1}^{9} x_{ijk} = 1 \quad \forall j,k = 1,\ldots,9
$$

**Cada número aparece exatamente uma vez por submatriz 3×3:**
Para cada submatriz $s = 1,\ldots,9$ e cada número $k = 1,\ldots,9$:

$$
\sum_{(i,j) \in B_s} x_{ijk} = 1
$$

onde $B_s$ é o conjunto de células da submatriz $s$.

---

**Valores pré-atribuídos:**
Para cada célula $(i,j)$ que já contém um valor $v$ no Sudoku inicial:

$$
x_{ijv} = 1
$$

---

**Formulação IP completa:**
Encontrar $x_{ijk}$ tal que:

$$
\sum_{k=1}^{9} x_{ijk} = 1 \quad \forall i,j = 1,\ldots,9
$$

$$
\sum_{j=1}^{9} x_{ijk} = 1 \quad \forall i,k = 1,\ldots,9
$$

$$
\sum_{i=1}^{9} x_{ijk} = 1 \quad \forall j,k = 1,\ldots,9
$$

$$
\sum_{(i,j) \in B_s} x_{ijk} = 1 \quad \forall s = 1,\ldots,9, \quad \forall k = 1,\ldots,9
$$

$$
x_{ijv} = 1 \quad \text{para cada célula pré-atribuída } (i,j) \text{ com valor } v
$$

$$
x_{ijk} \in \{0,1\} \quad \forall i,j,k = 1,\ldots,9
$$

---

**Definição das submatrizes 3×3:**
As submatrizes $B_s$ são definidas como:

- $B_1$: células $(i,j)$ com $i = 1,2,3$ e $j = 1,2,3$
- $B_2$: células $(i,j)$ com $i = 1,2,3$ e $j = 4,5,6$
- $B_3$: células $(i,j)$ com $i = 1,2,3$ e $j = 7,8,9$
- $B_4$: células $(i,j)$ com $i = 4,5,6$ e $j = 1,2,3$
- $B_5$: células $(i,j)$ com $i = 4,5,6$ e $j = 4,5,6$
- $B_6$: células $(i,j)$ com $i = 4,5,6$ e $j = 7,8,9$
- $B_7$: células $(i,j)$ com $i = 7,8,9$ e $j = 1,2,3$
- $B_8$: células $(i,j)$ com $i = 7,8,9$ e $j = 4,5,6$
- $B_9$: células $(i,j)$ com $i = 7,8,9$ e $j = 7,8,9$


In [32]:
# Tabuleiro inicial de exemplo (0 = célula vazia)
initial_board = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

print("Tabuleiro inicial:")
for row in initial_board:
    print(' '.join(str(x) if x != 0 else '.' for x in row))

# Criar problema de factibilidade
prob = pl.LpProblem("Sudoku", pl.LpMinimize)

# Variáveis de decisão
x = pl.LpVariable.dicts("x",
                         [(i, j, k) for i in range(9) for j in range(9) for k in range(1, 10)],
                         cat='Binary')

# Função objetivo dummy
prob += 0

# Restrição: cada célula tem exatamente um valor
for i in range(9):
    for j in range(9):
        prob += pl.lpSum(x[(i, j, k)] for k in range(1, 10)) == 1

# Restrição: cada número aparece exatamente uma vez por linha
for i in range(9):
    for k in range(1, 10):
        prob += pl.lpSum(x[(i, j, k)] for j in range(9)) == 1

# Restrição: cada número aparece exatamente uma vez por coluna
for j in range(9):
    for k in range(1, 10):
        prob += pl.lpSum(x[(i, j, k)] for i in range(9)) == 1

# Restrição: cada número aparece exatamente uma vez por submatriz 3x3
for block_row in range(3):
    for block_col in range(3):
        for k in range(1, 10):
            prob += pl.lpSum(x[(i, j, k)]
                             for i in range(block_row*3, block_row*3+3)
                             for j in range(block_col*3, block_col*3+3)) == 1

# Valores pré-atribuídos
for i in range(9):
    for j in range(9):
        if initial_board[i][j] != 0:
            k = initial_board[i][j]
            prob += x[(i, j, k)] == 1

# Resolver
prob.solve()

if pl.LpStatus[prob.status] == 'Optimal':
    print("\nSolução encontrada!")

    # Construir tabuleiro solução
    solution = [[0 for _ in range(9)] for _ in range(9)]
    for i in range(9):
        for j in range(9):
            for k in range(1, 10):
                if x[(i, j, k)].varValue > 0.5:
                    solution[i][j] = k

    # Imprimir solução
    print("\nSolução:")
    for i in range(9):
        row = []
        for j in range(9):
            row.append(str(solution[i][j]))
        print(' '.join(row))
else:
    print("\nNenhuma solução encontrada!")

Tabuleiro inicial:
5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9

Solução encontrada!

Solução:
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
